<a href="https://colab.research.google.com/github/totomartin28/buenas/blob/main/Copy_of_Codigo_Sofi_Colapsado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scrapeador de resoluciones de boletin oficial
Las URL del boletin oficial poseen una estructura casi unica, donde el unico elemento que tiene que variar para acceder a una pagina es el numero de resolución. Lo que hace nuestro codigo es aprovecharse de esta estructura para crear una lista con todas las URL del boletin oficial durante la era Alberto Fernandez. De esta manera, podemos utilizar Beautiful Soup (BS) para extraer el codigo fuente de cada pagina, y extraer de tal codigo fuente los elementos de texto que conforman cada resolución

In [ ]:
urls = []
inicio = 223503 #primer numero del boletin oficial
fin = 254161

for num in range(inicio, fin + 1):
    url = f"https://www.boletinoficial.gob.ar/detalleAviso/primera/{num}/20231113" #Estructura basica de cada URL. Lo que hace nuestro codigo es recorrer todos los numeros que le dimos en el intervalo inicio-fin. Numero en order, no aleatoriado
    urls.append(url)

Lo mas probable es que nuestra lista contenga un numero enorme de URLS a scrapear. Esto puede llegar a ser un problema a la hora de scraper tal lista con BS, ya que puede terminar en errores timeout (que nuestra sesión en Colab expire). Por ello, dividiremos nuestra lista de urls en distintas sublistas para poder manejarlas mejor.

In [ ]:
def dividir_lista(lista, partes):
    tamaño_sublista = len(lista) // partes
    sublistas = [lista[i:i+tamaño_sublista] for i in range(0, len(lista), tamaño_sublista)]
    return sublistas
partes = 4 #Especificamos el numero de sublistas que queremos producir
sublistas = dividir_lista(urls, partes)

Una vez tenemos nuestra sublistas, podemos empezar a scrapear

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

lista_urls = sublistas[0] #Definimos elemento "lista_urls", que sera cada sublista que querramos scrapear. Esto deberemos ir cambiandolo manualmente para evitar errores de timeout. Pensa que
                          #una sublista de 7000 elementos tarda aproximadamente 3 horas.

def obtener_contenido(url): #Definimos una función que extraiga el codigo fuente de cada url
    response = requests.get(url)
    if response.status_code == 200: #Chequea si podemos acceder a la pagina (codigo 200 significa que esta todo ok)
        return response.text #Extrae codigo fuente
    else:
        print("Error al obtener la página. Código de estado:", response.status_code)
        return None

def extraer_texto_detalle_aviso(url): #Ahora que tenemos el codigo fuente, queremos extraer aquellas partes que se encuentran dentro del elemento "detalleAviso", ya que dentro de este esta almacenado el texto de cada resolución
    contenido = obtener_contenido(url) #Usamos la función obtener_contenido y almacenamos el codigo fuente de cada url en el elemento contenido
    if contenido:
        soup = BeautifulSoup(contenido, 'html.parser')
        elemento = soup.find(id="detalleAviso") #Buscamos el elemento "detalleAviso"
        if elemento:
            return elemento.get_text() #Extraemos el texto del elemento detalleAviso
        else:
            print("No se encontró el elemento con id 'detalleAviso'")
    return None

textos = [] #Creamos una lista vacia donde almacenaremos los textos de cada resolución

for url in lista_urls:
    texto = extraer_texto_detalle_aviso(url) #Aplicamos la función extraer_texto_detalle_aviso por sobre cada url de nuestra lista lista_urls. Almacenamos el texto del boletin oficial en un elemento texto
    textos.append(texto) #Metemos cada elemento texto individual en la lista de textos que creamos con anterioridad

# Crear un DataFrame con los textos extraídos
df_urls = pd.DataFrame({'URL': lista_urls, 'Texto': textos}) #Creamos un dataframe que tenga dos columnas: una es la url de la resolución del boletin oficial, y la otra es el contenido de tal url (la resolución en si)

# Mostrar el DataFrame
df_urls.to_excel("Boletin_Oficial_sociedades_y_avisos_judiciales.xlsx", index=False) #Convertimos este dataframe en un archivo excel

from google.colab import files
files.download('Boletin_Oficial_sociedades_y_avisos_judiciales.xlsx') #Descargamos el archivo excel

KeyboardInterrupt: 

#Trabajando resoluciones scrapeadas
Una vez tengamos todas nuestras resoluciones de interes, podemos empezar a trabajar con cada una de ellas para extraer nuestros datos de interes

In [1]:
import pandas as pd
def importar_y_asignar_nombres(nombres_archivos): #Función para importar cada archivo de excel de nuestro entorno y leerlos como dataframes
    dfs = {}
    for nombre_archivo in nombres_archivos:
        try:
            nombre_df = nombre_archivo.split('.')[0]
            dfs[nombre_df] = pd.read_excel(nombre_archivo)
        except Exception as e:
            print(f"Error al importar el archivo {nombre_archivo}: {e}")
    return dfs

nombres_archivos = [ #Otorgamos los nombres de todos nuestros archivos excel que contengan las resoluciones, y los metemos en la lista "nombres_archivos"
    "Boletin_Oficial_2023.xlsx",
    "Boletin_Oficial_2023(1).xlsx",
    "Boletin_Oficial_16062022_0712.xlsx",
    "Boletin_Oficial_20191210_20211210_1.xlsx",
    "Boletin_Oficial_20191210_20211210_2.xlsx",
    "Boletin_Oficial_20191210_20211210_3.xlsx",
    "Boletin_Oficial_20191210_20211210_4.xlsx",
    "Boletin_Oficial_20191210_20211210_5.xlsx",
    "Boletin_Oficial_extension_suplemento_1.xlsx",
    "Boletin_Oficial_extension_suplemento_2.xlsx",
    "Boletin_Oficial_extension_suplemento_3.xlsx",
    "Boletin_Oficial_extension_suplemento_4.xlsx",
    "Boletin_Oficial_extension_suplemento_5.xlsx",
    "Boletin_Oficial_extension_suplemento_6.xlsx",
    "Boletin_Oficial_ley_de_ministerios.xlsx",
    "Parrafos_diciembre_enero_febrero.xlsx",
    "df_urls_textos.xlsx",
    "Boletin_Oficial_ley_de_ministerios_perdidos.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_0.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_1.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_2.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_3.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_4.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_5.xlsx",
    "Boletin_Oficial_extension_suplemento_perdidos_6.xlsx"
]

dataframes = importar_y_asignar_nombres(nombres_archivos) #Aplicamos la función importar_y_asignar_nombres a la lista "nombres_archivos"
df_unificado = pd.concat(dataframes.values(), ignore_index=True) #Concatenamos todos los df en uno solo

In [2]:
df_unificado = df_unificado.drop_duplicates() #Eliminamos elementos duplicados (resoluciones que esten mas de una vez por error humano)

In [3]:
df_unificado

,URL,Texto
0,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nDISTRIBUCIÓN DE ENERGÍA ELÉCTRICA Y GAS NA...
1,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nPROCEDIMIENTOS ADMINISTRATIVOS\nDecreto 81...
2,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nMINISTERIO DE DESARROLLO SOCIAL\nDecisión ...
3,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nMINISTERIO DE DESARROLLO SOCIAL\nDecisión ...
4,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nMINISTERIO DE SEGURIDAD\nDecisión Administ...
...,...,...
289605,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nSURFATEC S.A.\nREFORMA SA\n\n\n CUIT 30-71...
289606,https://www.boletinoficial.gob.ar/detalleAviso...,\n\n3 MOLINOS S.R.L.\nCONTRATO SRL\n\n\n Por e...
289607,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nAGZ URBANA S.R.L.\nCONTRATO SRL\n\n\n Por ...
289608,https://www.boletinoficial.gob.ar/detalleAviso...,\n\nANZECO S.R.L.\nCONTRATO SRL\n\n\n COMPLEME...


In [4]:
parrafos_completos=df_unificado["Texto"].tolist() #Extraemos la columna "Texto" y la convertimos en una lista. De esta forma, esta lista contendra todas las resoluciones del boletin oficial

In [5]:
import math
parrafos_completos = [x for x in parrafos_completos if not (isinstance(x, float) and math.isnan(x))] #Eliminamos los elementos nan de esta lista (urls que no poseian elemento "detalleAviso")

In [6]:
tipo_organismo = [parrafo.split('\n')[2] for parrafo in parrafos_completos] #Extraemos los organimos de cada resolución. El nombre de cada organismo esta en la tercera linea de cada resolución.
                                                                            #Por ello, dividimos el texto por cada salto de parrafo (cada linea), y extraemos el contenido de la tercer linea. Este es el
                                                                            #nombre de cada organismo, el cual metemos en la lista "tipo_organismo"

numero_resolucion = [parrafo.split('\n')[3] for parrafo in parrafos_completos] #Aplicamos la misma logica pero para extraer el numero de resolución. Esta se encuentra en la cuarta linea

Ahora tenemos el texto, los numeros de resolución y los nombres de cada organismo. Sin embargo, el texto sigue estando sucio al encontrarse en formato HTML, lo que puede traer subsecuentes problemas. En este apartado, "limpiamos" todos los elementos de sus respectivas listas, eliminando los saltos de linea (\n) y los elementos "\xa0"

In [7]:
tipo_organismo_procesado = []
for texto in tipo_organismo: #Reemplazamos saltos de linea por espacios en blanco para cada texto del boletin oficial, y lo metemos en la lista tipo_organismo_procesado
    if isinstance(texto, str) and texto is not None:
        texto_sin_saltos = texto.replace("\n", " ")
        tipo_organismo_procesado.append(texto_sin_saltos)

for i in range(len(tipo_organismo_procesado)): #De la lista tipo_organismo_procesado, tambien limpiamos los elementos "\xa0"
    tipo_organismo_procesado[i] = tipo_organismo_procesado[i].replace('\xa0', ' ')

numero_resolucion_procesado = []
for texto in numero_resolucion:
    if isinstance(texto, str) and texto is not None:
        texto_sin_saltos = texto.replace("\n", " ")
        numero_resolucion_procesado.append(texto_sin_saltos)

for i in range(len(numero_resolucion_procesado)):

    numero_resolucion_procesado[i] = numero_resolucion_procesado[i].replace('\xa0', ' ')

parrafos_completos_procesado = []
for texto in parrafos_completos:
    if isinstance(texto, str) and texto is not None:
        texto_sin_saltos = texto.replace("\n", " ")
        parrafos_completos_procesado.append(texto_sin_saltos)

for i in range(len(parrafos_completos_procesado)):
    parrafos_completos_procesado[i] = parrafos_completos_procesado[i].replace('\xa0', ' ')

Con nuestros elementos de interes ya limpios, los metemos en un nuevo dataframe al que llamaremos "df_procesado"

In [8]:
df_procesado=pd.DataFrame({"Texto Resolución": parrafos_completos_procesado, "Jurisdicción": tipo_organismo_procesado, "Numero Resolución": numero_resolucion_procesado})

Una vez que tenemos las resoluciones "limpias", podemos empezar a extraer elementos de ellas. Primero, extraeremos las fechas de cada una utilizando el paquete regex.

In [9]:
import re #Importamos paquete regex
import numpy as np

parrafos=parrafos_completos_procesado #Creamos lista de parrafos que vamos a usar localmente en esta celda

def encontrar_fecha(texto): #Definimos función para encontrar fecha
    patron = r'\b\d{2}/\d{2}/\d{4}\b' #Patron para encontrar fecha de regex
    coincidencias = re.findall(patron, texto) #Buscamos todas las coincidencias que haya para cada resolución
    return coincidencias[0] if coincidencias else None #Nos quedamos con la primer coincidencia que encuentre para cada resolución, la cual siempre sera la fecha de la propia resolución

fechas_en_parrafos = [] #Definimos lista donde seran almacenadas todas las fechas de los parrafos
for parrafo in parrafos:
    if isinstance(parrafo, str):
        fecha = encontrar_fecha(parrafo) #Aplicamos la función "encontrar_fecha" por sobre cada parrafo del boletin oficial. Creamos elemento "fecha"
        fechas_en_parrafos.append(fecha) #Metemos cada elemento fecha a la lista "fechas_en_parrafos"
    else:
        fechas_en_parrafos.append(None)

df_procesado["Fecha de Resolución"]= fechas_en_parrafos #Agregamos la lista "fechas_en_parrafos" como una columna a nuestro dataframe

#Filtrando resoluciones de interes

In [10]:
lista_sin_duplicados = list(set(df_procesado["Jurisdicción"].tolist())) #Convertimos la columna de jurisdicción en una lista

def filtrar_elementos(lista): #Nos quedamos unicamente con aquellas observaciones que no arranquen con alguna de estas palabras para la columna "Jurisdicción"
    palabras_excluidas = ["MINISTERIO", "SECRETARÍA", "JEFATURA"]
    elementos_filtrados = []

    for elemento in lista:
        agregar_elemento = True
        for palabra in palabras_excluidas:
            if elemento.startswith(palabra):
                agregar_elemento = False
                break
        if agregar_elemento:
            elementos_filtrados.append(elemento)

    return elementos_filtrados

lista = lista_sin_duplicados
lista_filtrada = filtrar_elementos(lista)

In [11]:
#Metemos aquellos organismos que arrancan con secretaría (u otra palabra) que aun asi son irrelevantes, y sacamos de esta a organismos relevantes (como el INAI)
lista_filtrada.remove("INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS")
lista_filtrada.remove("INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOMÍA SOCIAL")
lista_filtrada.append("SECRETARÍA LEGAL Y TÉCNICA")
lista_filtrada.append("SECRETARÍA GENERAL")
lista_filtrada.append("SECRETARÍA DE COMUNICACIÓN Y PRENSA")
lista_filtrada.append("SECRETARÍA GENERAL DIRECCIÓN GENERAL DE CEREMONIAL")
lista_filtrada.append("SECRETARÍA DE POLÍTICAS INTEGRALES SOBRE DROGAS DE LA NACIÓN ARGENTINA")
lista_filtrada.append("SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO")
lista_filtrada.append("NAVARRO FLORIA, Juan G.")
organismos = lista_filtrada

df_procesado = df_procesado[~df_procesado['Jurisdicción'].str.upper().isin(organismos)] #Quitamos aquellos organismos que se encuentren en la lista "organismos"

In [12]:
df_procesado

,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución
2,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1224/2022,07/12/2022
3,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1225/2022,07/12/2022
4,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1226/2022,07/12/2022
5,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1227/2022,07/12/2022
6,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",Decisión Administrativa 1223/2022,07/12/2022
...,...,...,...,...
239388,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCI...",Disposición 40/2023,02/02/2023
239389,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCI...",Disposición 39/2023,02/02/2023
239390,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCI...",Disposición 38/2023,02/02/2023
239391,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCI...",Resolución 1093/2021,02/02/2023


#Dividimos las resoluciones en sus articulos individuales

In [13]:
#Dividimos el texto en sus articulos individuales
subparrafos = []
parrafos=df_procesado["Texto Resolución"].tolist()
for parrafo in parrafos:
    subparrafo_actual = []
    for subparrafo in parrafo.split("ARTÍCULO")[1:]:
        subparrafo_actual.append("ARTÍCULO" + subparrafo)
    subparrafos.append(subparrafo_actual)

df_procesado["Articulos"]=subparrafos
df_procesado = df_procesado.explode('Articulos')

#Categorizando resoluciones del Boletin Oficial

In [14]:
#Categorizamos cada articulo por las palabras que se encuentran presentes en el, a modo tal de saber si estamos trabajando con una designación, una renuncia o una prorroga
import numpy as np

df_procesado = df_procesado.dropna(subset=['Articulos'])

# Definir las palabras clave
palabras_clave = ["Desígnase", "Dase por designado", "Dase por designada", "Dase por asignado", "Dase por asignada", "Dáse por designado",
                  "Dáse por designada", "dáse por designado", "dáse por designada", "Designaciones", "designaciones"]

# Función para buscar palabras clave en un texto
def buscar_palabras_clave(texto):
    for palabra in palabras_clave:
        if palabra in texto:
            return "hit"
    return np.nan

# Crear la nueva columna 'resultado'
df_procesado['Designación'] = df_procesado['Articulos'].apply(buscar_palabras_clave)


#Renuncias
palabras_clave = ["renuncia", "renuncias"]
df_procesado['Renuncia'] = df_procesado['Articulos'].apply(buscar_palabras_clave)

#Prorrogas
palabras_clave = ["Prórroga", "Dar por prorrogadas", "Prorróguese", "Dase por prorrogada", "Prorrogar", "Prorróga", "prorrogar", "prorrogadas", "prorróguese"]
df_procesado['Prorroga'] = df_procesado['Articulos'].apply(buscar_palabras_clave)

Recategorizamos en base a las primeras 20 palabras del articulo para evitar errores de codeo respecto a las prorrogas que contienen la palabra "Designación"

In [15]:
lista_prueba=df_procesado["Articulos"].tolist()
lista_prueba_0=lista_prueba

In [16]:
def extraer_primera_palabra(texto):
    if pd.isna(texto):
        return np.nan
    return texto.split()[:20]

primeras_palabras = [extraer_primera_palabra(texto) for texto in lista_prueba]
print(primeras_palabras)

In [ ]:
def unir_frases(sublista):
    return ' '.join(sublista)

# Aplicar la función a cada sublista en la lista de sublistas
lista_unida = [unir_frases(sublista) for sublista in primeras_palabras]

print(lista_unida)

In [ ]:
df_procesado["Primeras Palabras"]=lista_unida

In [ ]:
df=df_procesado

In [ ]:
#Aplicamos la función de clasificación a las primeras 20 palabras de cada articulo
import numpy as np


# Definir las palabras clave
palabras_clave = ["Desígnase", "Dase por designado", "Dase por designada", "Dase por asignado", "Dase por asignada", "Dáse por designado",
                  "Dáse por designada", "dáse por designado", "dáse por designada"]

# Función para buscar palabras clave en un texto
def buscar_palabras_clave(texto):
    for palabra in palabras_clave:
        if palabra in texto:
            return "hit"
    return np.nan

# Crear la nueva columna 'resultado'
df['Designación_0'] = df['Primeras Palabras'].apply(buscar_palabras_clave)


#Renuncias
palabras_clave = ["renuncia", "renuncias", "DEJESE SIN EFECTO"]
df['Renuncia_0'] = df['Primeras Palabras'].apply(buscar_palabras_clave)

#Prorrogas
palabras_clave = ["Prórroga", "Dar por prorrogadas", "Prorróguese", "Dase por prorrogada", "Prorrogar", "Prorróga", "prorrogar", "prorrogadas", "prorróguese"]
df['Prorroga_0'] = df['Primeras Palabras'].apply(buscar_palabras_clave)

condicion = df['Designación_0'].notna() & df['Prorroga_0'].notna()

# Asignar NaN a la columna A donde la condición es verdadera
df.loc[condicion, 'Designación_0'] = np.nan

df = df.dropna(how='all', subset=['Designación_0', 'Renuncia_0', 'Prorroga_0']) #Eliminamos las resoluciones que no contengan
#una de las palabras clave, ya que no tratan eventos de interes

In [ ]:
def transformar_valor(valor):
    return "Designación"
df['Designación_0'] = df['Designación_0'].apply(lambda x: transformar_valor(x) if pd.notna(x) else x)

def transformar_valor(valor):
    return "Renuncia"
df['Renuncia_0'] = df['Renuncia_0'].apply(lambda x: transformar_valor(x) if pd.notna(x) else x)

def transformar_valor(valor):
    return "Prorroga"
df['Prorroga_0'] = df['Prorroga_0'].apply(lambda x: transformar_valor(x) if pd.notna(x) else x)

#Categorizamos los articulos en la columna "Clasificación_resolución"
df['Clasificación_resolución'] = np.nan
df['Clasificación_resolución'] = df['Clasificación_resolución'].fillna(df['Designación_0']).fillna(df['Renuncia_0']).fillna(df['Prorroga_0'])
df

In [ ]:
df.to_excel("Articulos de interes.xlsx", index=False)
from google.colab import files
files.download('Articulos de interes.xlsx')

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
data=pd.read_excel("Articulos de interes.xlsx")
df=pd.DataFrame(data)

#Extrayendo fecha de articulos

In [25]:
#Las fechas dentro de los articulos estan en formato texto. Esta celda lo que hace es buscar estas fechas, y transformarlas al formato DD/MM/YYYY
patron_fecha = r'\b(\d{1,2})(?:º|°)? de ([^\W\d_]{3,}) de (\d{4})\b|\b(\d{1,2})(?:º|°)? de ([^\W\d_]{3,}) (?:del )?(\d{4})\b'
fechas_encontradas_oraciones = []


oraciones = df["Articulos"].tolist()
for oracion in oraciones:
    fechas_mixtas = re.findall(patron_fecha, oracion)
    fechas_encontradas_oraciones.append(fechas_mixtas)

def eliminar_elementos_vacios(lista_tuplas):
    return [tuple(filter(lambda x: x != '', tupla)) for tupla in lista_tuplas if tupla]
fechas_encontradas_oraciones = [eliminar_elementos_vacios(tupla) for tupla in fechas_encontradas_oraciones]

def formatear_fecha(fecha_tuple):
    if fecha_tuple:
        dia, mes, año = fecha_tuple
        fecha_str = f"{dia} de {mes} de {año}"
        return fecha_str
    return np.nan
fechas_formateadas = [[formatear_fecha(fecha) for fecha in lista_fechas] for lista_fechas in fechas_encontradas_oraciones]

def convertir_fecha(texto_fecha):
    meses = {'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',
             'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'}
    if isinstance(texto_fecha, str):
        partes = texto_fecha.split(' ')
        dia = partes[0]
        mes = meses.get(partes[2].lower())
        año = partes[4]
        if dia and mes and año:
            fecha_formateada = f"{dia}/{mes}/{año}"
            return fecha_formateada
        else:
            return np.nan
    else:
        return np.nan
fechas_convertidas = [[convertir_fecha(fecha) for fecha in lista_fechas] for lista_fechas in fechas_formateadas]
df["Fechas de Designación"]=fechas_convertidas
df#Las fechas dentro de los articulos estan en formato texto. Esta celda lo que hace es buscar estas fechas, y transformarlas al formato DD/MM/YYYY
patron_fecha = r'\b(\d{1,2})(?:º|°)? de ([^\W\d_]{3,}) de (\d{4})\b|\b(\d{1,2})(?:º|°)? de ([^\W\d_]{3,}) (?:del )?(\d{4})\b'
fechas_encontradas_oraciones = []


oraciones = df["Articulos"].tolist()
for oracion in oraciones:
    fechas_mixtas = re.findall(patron_fecha, oracion)
    fechas_encontradas_oraciones.append(fechas_mixtas)

def eliminar_elementos_vacios(lista_tuplas):
    return [tuple(filter(lambda x: x != '', tupla)) for tupla in lista_tuplas if tupla]
fechas_encontradas_oraciones = [eliminar_elementos_vacios(tupla) for tupla in fechas_encontradas_oraciones]

def formatear_fecha(fecha_tuple):
    if fecha_tuple:
        dia, mes, año = fecha_tuple
        fecha_str = f"{dia} de {mes} de {año}"
        return fecha_str
    return np.nan
fechas_formateadas = [[formatear_fecha(fecha) for fecha in lista_fechas] for lista_fechas in fechas_encontradas_oraciones]

def convertir_fecha(texto_fecha):
    meses = {'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',
             'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'}
    if isinstance(texto_fecha, str):
        partes = texto_fecha.split(' ')
        dia = partes[0]
        mes = meses.get(partes[2].lower())
        año = partes[4]
        if dia and mes and año:
            fecha_formateada = f"{dia}/{mes}/{año}"
            return fecha_formateada
        else:
            return np.nan
    else:
        return np.nan
fechas_convertidas = [[convertir_fecha(fecha) for fecha in lista_fechas] for lista_fechas in fechas_formateadas]
df["Fechas de Designación"]=fechas_convertidas

Ahora tenemos una lista de fechas por articulo. Como los articulos suelen tener fechas que hacen referencia a resoluciones anteriores y que no tienen nada que ver con la designación/prorroga o renuncia del funcionario, nos quedamos solamente con la fecha que sea mas cercana a la fecha de publicación de la normativa en el Boletin Oficial

In [26]:
df['Fecha de Resolución'] = pd.to_datetime(df['Fecha de Resolución'], format='%d/%m/%Y')

# Función para encontrar la fecha más cercana
def fecha_mas_cercana(fechas, fecha_resolucion):
    if not fechas:  # Si la lista está vacía, devolver NaN
        return np.nan
    fechas = pd.to_datetime(fechas, format='%d/%m/%Y')
    diferencia = abs(fechas - fecha_resolucion)
    return fechas[diferencia.argmin()]

# Aplicar la función a cada fila
df['Fecha Más Cercana'] = df.apply(lambda row: fecha_mas_cercana(row['Fechas de Designación'], row['Fecha de Resolución']), axis=1)

fecha_target = pd.to_datetime('10/12/2019', format='%d/%m/%Y')

# Reemplazar las fechas anteriores a la fecha objetivo con NaT
df['Fecha Más Cercana'] = df['Fecha Más Cercana'].where(df['Fecha Más Cercana'] >= fecha_target, pd.NaT) #Si la fecha resultante es anterior al 12/10/2019,
                                                                                                          #se le asigna a la observación una fecha nan
df['Fecha Más Cercana'] =df['Fecha Más Cercana'].fillna(df['Fecha de Resolución']) #En caso de fecha nan para una observación, se le asigna la fecha de publicación de la resolución
df

<ipython-input-26-66def45b828a>:9: FutureWarning: The behavior of TimedeltaIndex.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  return fechas[diferencia.argmin()]
<ipython-input-26-66def45b828a>:9: FutureWarning: The behavior of TimedeltaIndex.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  return fechas[diferencia.argmin()]


,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Designación,Renuncia,Prorroga,Primeras Palabras,Designación_0,Renuncia_0,Prorroga_0,Clasificación_resolución,Fechas de Designación,Fecha Más Cercana
0,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1224/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[1/09/2022],2022-09-01
1,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1225/2022,2022-12-07,ARTÍCULO 1°.- Dase por designado con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designado con carácter ...,Designación,NaN,NaN,Designación,[1/09/2022],2022-09-01
2,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1226/2022,2022-12-07,ARTÍCULO 1º.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1º.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[1/10/2022],2022-10-01
3,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1227/2022,2022-12-07,ARTÍCULO 1º.- Desígnase con carácter transitor...,hit,NaN,NaN,ARTÍCULO 1º.- Desígnase con carácter transitor...,Designación,NaN,NaN,Designación,[],2022-12-07
4,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",Decisión Administrativa 1223/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[14/11/2022],2022-11-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9565,MINISTERIO DE ECONOMÍA Resolución 69/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 69/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 29...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 29...",NaN,NaN,Prorroga,Prorroga,"[29/11/2022, 3/12/2008]",2022-11-29
9566,MINISTERIO DE ECONOMÍA Resolución 71/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 71/2023,2023-02-02,"ARTÍCULO 1°.- Danse por prorrogadas, desde el ...",hit,NaN,hit,"ARTÍCULO 1°.- Danse por prorrogadas, desde el ...",NaN,NaN,Prorroga,Prorroga,[3/12/2008],2023-02-02
9567,MINISTERIO DE ECONOMÍA Resolución 72/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 72/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,Prorroga,Prorroga,"[2/09/2022, 3/12/2008]",2022-09-02
9568,MINISTERIO DE ECONOMÍA Resolución 73/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 73/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 14...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 14...",NaN,NaN,Prorroga,Prorroga,"[14/12/2022, 3/12/2008]",2022-12-14


#Identificando resoluciones con anexo
Como las resoluciones con anexo tienen particularidades propias, debemos trabajarlas de forma separada

In [27]:
#Creamos una columna que nos indique si el articulo del boletin oficial hace referencia a un anexo. De hacer referencia a uno, la observación es trasladada a un nuevo df
df['Anexo'] = False
df.loc[df['Articulos'].str.contains('anexo', case=False), 'Anexo'] = True
df_anexos=df[df['Anexo']==True]

###Extrayendo DNI de las resoluciones con anexos

In [28]:
patron = r'\b\d{1,3}(?:\.\d{3}){2}\b'
numeros_dict = {}

oraciones = df_anexos["Texto Resolución"].tolist() #Como los DNI suelen estar en la sección de "considerando", y no en los articulos, extraemos los DNI presentes en toda la resolución
oraciones = list(map(str, oraciones))

for index, oracion in enumerate(oraciones):
    numeros_encontrados = re.findall(patron, oracion)
    if numeros_encontrados:
        numeros_sin_punto = [numero.replace('.', '') for numero in numeros_encontrados]
        numeros_dict[index] = numeros_sin_punto

numeros_extraidos = [np.nan] * len(df_anexos)
for index, numeros in numeros_dict.items():
    numeros_extraidos[index] = numeros
df_anexos["DNI Extraido"] = numeros_extraidos
df_anexos=df_anexos.explode('DNI Extraido')
df_anexos

<ipython-input-28-2040d3f43feb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexos["DNI Extraido"] = numeros_extraidos


,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Designación,Renuncia,Prorroga,Primeras Palabras,Designación_0,Renuncia_0,Prorroga_0,Clasificación_resolución,Fechas de Designación,Fecha Más Cercana,Anexo,DNI Extraido
16,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBIT...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Resolución 382/2022,2022-12-12,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",hit,NaN,hit,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",NaN,NaN,Prorroga,Prorroga,[],2022-12-12,True,21443543
16,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBIT...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Resolución 382/2022,2022-12-12,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",hit,NaN,hit,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",NaN,NaN,Prorroga,Prorroga,[],2022-12-12,True,10965595
16,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBIT...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Resolución 382/2022,2022-12-12,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",hit,NaN,hit,"ARTÍCULO 1°.- Dese por prorrogadas, desde la f...",NaN,NaN,Prorroga,Prorroga,[],2022-12-12,True,17331396
24,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIB...,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIBLE,Resolución 534/2022,2022-12-13,ARTÍCULO 1°.- Dánse por prorrogadas las design...,hit,NaN,hit,ARTÍCULO 1°.- Dánse por prorrogadas las design...,NaN,NaN,Prorroga,Prorroga,[31/03/2020],2020-03-31,True,NaN
35,MINISTERIO DE ECONOMÍA Resolución 1013/2022 ...,MINISTERIO DE ECONOMÍA,Resolución 1013/2022,2022-12-14,"ARTÍCULO 1°.- Dase por prorrogada, desde el 6 ...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 6 ...",NaN,NaN,Prorroga,Prorroga,"[6/09/2022, 3/12/2008]",2022-09-06,True,26250691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9534,MINISTERIO DE TURISMO Y DEPORTES Resolución ...,MINISTERIO DE TURISMO Y DEPORTES,Resolución 14/2023,2023-01-25,"ARTÍCULO 2°.- Desígnase, a partir del dictado ...",hit,NaN,NaN,"ARTÍCULO 2°.- Desígnase, a partir del dictado ...",Designación,NaN,NaN,Designación,[],2023-01-25,True,NaN
9562,MINISTERIO DE ECONOMÍA Resolución 65/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 65/2023,2023-02-02,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",hit,NaN,hit,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",NaN,NaN,Prorroga,Prorroga,[3/12/2008],2023-02-02,True,26348017
9562,MINISTERIO DE ECONOMÍA Resolución 65/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 65/2023,2023-02-02,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",hit,NaN,hit,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",NaN,NaN,Prorroga,Prorroga,[3/12/2008],2023-02-02,True,29535811
9563,MINISTERIO DE ECONOMÍA Resolución 67/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 67/2023,2023-02-02,"ARTÍCULO 1°.- Danse por prorrogadas, a partir ...",hit,NaN,hit,"ARTÍCULO 1°.- Danse por prorrogadas, a partir ...",NaN,NaN,Prorroga,Prorroga,[3/12/2008],2023-02-02,True,NaN


In [29]:
df_anexos_na=df_anexos[df_anexos['DNI Extraido'].isna()]
df_anexos_na.to_excel('df_anexos_na.xlsx', index=False) #Este df contiene todas las resoluciones que contienen anexos pero que no hacen referencia a ningun DNI.
                                                        #La guardamos para mas tarde seguir trabajando con estas resoluciones

from google.colab import files
files.download('df_anexos_na.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Trabajando resoluciones sin anexo

In [30]:
df_sin_anexos=df[df['Anexo']==False]

In [31]:
patron = r'\b\d{1,3}(?:\.\d{3}){2}\b'
numeros_dict = {}

oraciones = df_sin_anexos["Articulos"].tolist() #Como las resoluciones sin
oraciones = list(map(str, oraciones))

for index, oracion in enumerate(oraciones):
    numeros_encontrados = re.findall(patron, oracion)
    if numeros_encontrados:
        numeros_sin_punto = [numero.replace('.', '') for numero in numeros_encontrados]
        numeros_dict[index] = numeros_sin_punto

numeros_extraidos = [np.nan] * len(df_sin_anexos)
for index, numeros in numeros_dict.items():
    numeros_extraidos[index] = numeros
df_sin_anexos["DNI Extraido"] = numeros_extraidos
df_sin_anexos=df_sin_anexos.explode('DNI Extraido')
df_sin_anexos

<ipython-input-31-4c53ed41fc38>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_anexos["DNI Extraido"] = numeros_extraidos


,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Designación,Renuncia,Prorroga,Primeras Palabras,Designación_0,Renuncia_0,Prorroga_0,Clasificación_resolución,Fechas de Designación,Fecha Más Cercana,Anexo,DNI Extraido
0,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1224/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[1/09/2022],2022-09-01,False,33326011
1,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1225/2022,2022-12-07,ARTÍCULO 1°.- Dase por designado con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designado con carácter ...,Designación,NaN,NaN,Designación,[1/09/2022],2022-09-01,False,28417542
2,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1226/2022,2022-12-07,ARTÍCULO 1º.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1º.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[1/10/2022],2022-10-01,False,35796265
3,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1227/2022,2022-12-07,ARTÍCULO 1º.- Desígnase con carácter transitor...,hit,NaN,NaN,ARTÍCULO 1º.- Desígnase con carácter transitor...,Designación,NaN,NaN,Designación,[],2022-12-07,False,38070271
4,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",Decisión Administrativa 1223/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,hit,NaN,NaN,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,NaN,NaN,Designación,[14/11/2022],2022-11-14,False,24366003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,MINISTERIO DE ECONOMÍA Resolución 68/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 68/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,Prorroga,Prorroga,"[2/09/2022, 3/12/2008]",2022-09-02,False,22931181
9565,MINISTERIO DE ECONOMÍA Resolución 69/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 69/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 29...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 29...",NaN,NaN,Prorroga,Prorroga,"[29/11/2022, 3/12/2008]",2022-11-29,False,37481900
9567,MINISTERIO DE ECONOMÍA Resolución 72/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 72/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 2 ...",NaN,NaN,Prorroga,Prorroga,"[2/09/2022, 3/12/2008]",2022-09-02,False,33113342
9568,MINISTERIO DE ECONOMÍA Resolución 73/2023 RE...,MINISTERIO DE ECONOMÍA,Resolución 73/2023,2023-02-02,"ARTÍCULO 1°.- Dase por prorrogada, desde el 14...",NaN,NaN,hit,"ARTÍCULO 1°.- Dase por prorrogada, desde el 14...",NaN,NaN,Prorroga,Prorroga,"[14/12/2022, 3/12/2008]",2022-12-14,False,33309936


#Volviendo a unificar el DF

In [32]:
df_combined = pd.concat([df_sin_anexos, df_anexos], ignore_index=True) #Concatenamos ambos df de resoluciones con y sin anexos

In [33]:
df_combined=df_combined.drop_duplicates(subset=['DNI Extraido', 'Articulos']) #Eliminamos observaciones duplicadas

In [34]:
df_combined = df_combined.dropna(subset=['DNI Extraido']) #Eliminamos observaciones que no tengan ningun DNI

In [35]:
df=df_combined

#Eliminando palabras clave

In [36]:
df = df.drop(columns=['Designación', 'Renuncia',
       'Prorroga', 'Primeras Palabras', 'Designación_0', 'Renuncia_0',
       'Prorroga_0', 'Fechas de Designación'])

In [37]:
palabras_eliminar = ['representante titular',
                     'representantes titulares',
                     'representante suplente',
                     'Representante especial',
                     'beneficio emergente',
                     'unidad de implementación',
                     'con carácter ad honorem',
                     'núcleo de expertos',
                     'director de obra titular',
                     'director de obra suplente',
                     'vocales',
                     'vocal',
                     'consejo de administración',
                     'comisión nacional de',
                     'beneficio instituido',
                     'como representante del',
                     'como representante de',
                     'desígnase como representantes del',
                     'en el Directorio del',
                     'equipo técnico de trabajo',
                     'importe total de pesos',
                     'ad-honorem',
                     '"ad-honorem"',
                     'ad honorem',
                     'ad honórem',
                     'ad-honórem',
                     'miembros titulares',
                     'miembros suplentes',
                     'punto focal',
                     'chofer',
                     'mozo',
                     'delegado normalizador',
                     'delegada normalizadora',
                     'delegado',
                     'delegada',
                     'en representación de',
                     'Funcionario Interventor',
                     'funcionarios interventores',
                     'comisión evaluadora',
                     'empresa del estado',
                     'ad honorem',
                     'comité evaluador',
                     'asamblea ordinaria',
                     'nómina',
                     'Prorrógase el contrato',
                     'pesos'
                     ]


df = df[~df['Articulos'].str.contains('|'.join(palabras_eliminar), case=False, na=False)]

Ahora queremos filtrar las frases "organismo descentralizado" y "organismo desconcentrado", pero no queremos eliminar las observaciones correspondientes al INAI o al INAI

In [38]:
frases_a_buscar = ['instituto nacional de asociativismo y economía social', 'instituto nacional de asuntos indígenas', 'INAES', 'INAI']

# Inicializar la columna 'Contiene INAES o INAI' como False
df['Contiene INAES o INAI'] = False

# Actualizar la columna 'Contiene INAES o INAI' con True para las filas que contienen las frases
df.loc[df['Articulos'].str.contains('|'.join(frases_a_buscar), case=False, na=False), 'Contiene INAES o INAI'] = True

<ipython-input-38-4e076ac9909d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contiene INAES o INAI'] = False


In [39]:
df

,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Clasificación_resolución,Fecha Más Cercana,Anexo,DNI Extraido,Contiene INAES o INAI
0,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1224/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,2022-09-01,False,33326011,False
1,MINISTERIO DE DESARROLLO SOCIAL Decisión Adm...,MINISTERIO DE DESARROLLO SOCIAL,Decisión Administrativa 1225/2022,2022-12-07,ARTÍCULO 1°.- Dase por designado con carácter ...,Designación,2022-09-01,False,28417542,False
2,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1226/2022,2022-12-07,ARTÍCULO 1º.- Dase por designada con carácter ...,Designación,2022-10-01,False,35796265,False
3,MINISTERIO DE SEGURIDAD Decisión Administrat...,MINISTERIO DE SEGURIDAD,Decisión Administrativa 1227/2022,2022-12-07,ARTÍCULO 1º.- Desígnase con carácter transitor...,Designación,2022-12-07,False,38070271,False
4,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SO...","MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",Decisión Administrativa 1223/2022,2022-12-07,ARTÍCULO 1°.- Dase por designada con carácter ...,Designación,2022-11-14,False,24366003,False
...,...,...,...,...,...,...,...,...,...,...
10518,MINISTERIO DE TURISMO Y DEPORTES Resolución ...,MINISTERIO DE TURISMO Y DEPORTES,Resolución 215/2022,2022-05-03,"ARTÍCULO 1°.- Danse por prorrogadas, desde y h...",Prorroga,2022-05-03,True,25312301,False
10519,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIB...,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIBLE,Resolución 172/2022,2022-05-03,ARTÍCULO 1°.- Dánse por prorrogadas las design...,Prorroga,2020-03-31,True,25864469,False
10521,MINISTERIO DE SEGURIDAD Resolución 940/2023 ...,MINISTERIO DE SEGURIDAD,Resolución 940/2023,2023-12-14,ARTÍCULO 1°.- Desígnase al Doctor Ricardo José...,Designación,2023-12-14,True,27823686,False
10524,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBIT...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Resolución 449/2021,2021-12-10,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",Prorroga,2021-12-10,True,28507233,False


In [40]:
palabras_eliminar = ['organismo descentralizado', 'organismo desconcentrado']

# Filtrar las observaciones donde 'Contiene INAES o INAI' es False
mask_false = df['Contiene INAES o INAI'] == False
df_false = df[mask_false]

# Aplicar la función de eliminación de palabras a la columna B en el DataFrame filtrado
df_false_filtered = df_false[~df_false['Articulos'].str.contains('|'.join(palabras_eliminar), case=False, na=False)]

# Combinar las observaciones filtradas con las observaciones donde 'Contiene INAES o INAI' es True
df_result = pd.concat([df[~mask_false], df_false_filtered])
df=df_result

In [41]:
df = df.drop(columns=['Anexo', 'Contiene INAES o INAI'])

#Eliminando fechas de designación posterior a 10/12/2023

In [42]:
df

,Texto Resolución,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Clasificación_resolución,Fecha Más Cercana,DNI Extraido
47,INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS Decr...,INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS,Decreto 838/2022,2022-12-16,"ARTÍCULO 1º.- Dase por designado, a partir del...",Designación,2022-11-03,17432126
345,INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOM...,INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOMÍA...,Resolución 68/2023,2023-03-07,ARTÍCULO 1.- Desígnase a la agente Lic. Bárbar...,Designación,2023-03-07,30655760
359,INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOM...,INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOMÍA...,Decisión Administrativa 196/2023,2023-03-09,ARTÍCULO 1º.- Dase por designado con carácter ...,Designación,2023-03-01,28643563
501,INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS Deci...,INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS,Decisión Administrativa 332/2023,2023-04-24,ARTÍCULO 1º.- Dase por designada con carácter ...,Designación,2022-11-05,23116347
1128,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS De...,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,Decreto 671/2023,2023-12-05,ARTÍCULO 7°.- Acéptase la renuncia presentada ...,Renuncia,2023-12-10,17432126
...,...,...,...,...,...,...,...,...
10518,MINISTERIO DE TURISMO Y DEPORTES Resolución ...,MINISTERIO DE TURISMO Y DEPORTES,Resolución 215/2022,2022-05-03,"ARTÍCULO 1°.- Danse por prorrogadas, desde y h...",Prorroga,2022-05-03,25312301
10519,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIB...,MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIBLE,Resolución 172/2022,2022-05-03,ARTÍCULO 1°.- Dánse por prorrogadas las design...,Prorroga,2020-03-31,25864469
10521,MINISTERIO DE SEGURIDAD Resolución 940/2023 ...,MINISTERIO DE SEGURIDAD,Resolución 940/2023,2023-12-14,ARTÍCULO 1°.- Desígnase al Doctor Ricardo José...,Designación,2023-12-14,27823686
10524,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBIT...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Resolución 449/2021,2021-12-10,"ARTÍCULO 1°.- Danse por prorrogadas, desde la ...",Prorroga,2021-12-10,28507233


In [43]:
fecha_target = pd.to_datetime('10/12/2023', format='%d/%m/%Y')

mask = df['Clasificación_resolución'] == 'Designación'
df.loc[mask, 'Fecha de Resolución'] = df.loc[mask, 'Fecha de Resolución'].where(df.loc[mask, 'Fecha de Resolución'] <= fecha_target, pd.NaT)

df = df.dropna(subset=['Fecha de Resolución'])

In [44]:
#Reemp
fecha_target = pd.to_datetime('10/12/2023', format='%d/%m/%Y')
df['Fecha Más Cercana'] = df['Fecha Más Cercana'].where(df['Fecha Más Cercana'] <= fecha_target, pd.NaT)
df['Fecha Más Cercana'] =df['Fecha Más Cercana'].fillna(df['Fecha de Resolución'])

<ipython-input-44-bc50333c8099>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha Más Cercana'] = df['Fecha Más Cercana'].where(df['Fecha Más Cercana'] <= fecha_target, pd.NaT)
<ipython-input-44-bc50333c8099>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha Más Cercana'] =df['Fecha Más Cercana'].fillna(df['Fecha de Resolución'])


#Arreglando fechas de prorrogas

In [45]:
#Hubo problemas con las fechas de prorrogas por las fechas de los considerandos. Procedemos a poner fecha de decreto para las prorrogas.
df.loc[df['Clasificación_resolución'] == 'Prorroga', 'Fecha Más Cercana'] = pd.NaT
df['Fecha Más Cercana'] =df['Fecha Más Cercana'].fillna(df['Fecha de Resolución'])

<ipython-input-45-5f906756372d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha Más Cercana'] =df['Fecha Más Cercana'].fillna(df['Fecha de Resolución'])


#Ordenando cronologicamente los articulos

In [46]:
df = df.sort_values(by='Fecha de Resolución')

#Eliminamos resoluciones que no tengan DNI asociadas a ellas

In [47]:
df = df.dropna(subset=['DNI Extraido'])

#Creamos una colummna de identificación

In [48]:
df['identificacion'] = df.apply(lambda row: [row['Fecha Más Cercana'], row['Articulos'] ,row['Numero Resolución'], row['Jurisdicción'], row['Clasificación_resolución']], axis=1)

#Eliminamos variable "Texto Resolución" ya que es innecesaria

In [49]:
df=df.drop(columns=["Texto Resolución"])

#Creamos variable Designación_Flags.
En esta variable, las observaciones que sean designaciones son codificadas como 1, las renuncias como 0, y las prorrogas como 2

In [50]:
def transform_value(event):
    mapping = {'Designación': 1, 'Renuncia': 0, 'Prorroga': 2}
    return mapping.get(event, -1)

# Aplicar la función a la columna 'Clasificación_resolución'
df['Designación_Flags'] = df['Clasificación_resolución'].apply(transform_value)

In [51]:
#Creamos un diccionario donde quede asignado, a cada valor de la variable Designación_Flags, los valores de las demas variables
def create_dict(key, value_list):
    return {key: value_list}

# Aplicar la función y crear la nueva columna
df['Nuevo_Diccionario'] = df.apply(lambda row: create_dict(row['Designación_Flags'], row['identificacion']), axis=1)

df

,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Clasificación_resolución,Fecha Más Cercana,DNI Extraido,identificacion,Designación_Flags,Nuevo_Diccionario
2280,MINISTERIO DE RELACIONES EXTERIORES Y CULTO,Resolución 691/2019,2019-12-10,ARTÍCULO 1°.- Acéptase la renuncia del señor M...,Renuncia,2019-12-10,22053898,"[2019-12-10 00:00:00, ARTÍCULO 1°.- Acéptase l...",0,"{0: [2019-12-10 00:00:00, 'ARTÍCULO 1°.- Acépt..."
2247,"SECRETARÍA DE GOBIERNO DE CIENCIA, TECNOLOGÍA ...",Decreto 885/2019,2019-12-10,"ARTÍCULO 2º.- Acéptase, a partir del 10 de dic...",Renuncia,2019-12-10,23204720,"[2019-12-10 00:00:00, ARTÍCULO 2º.- Acéptase, ...",0,"{0: [2019-12-10 00:00:00, 'ARTÍCULO 2º.- Acépt..."
2248,SECRETARÍA DE GOBIERNO DE TRABAJO Y EMPLEO,Decreto 880/2019,2019-12-10,"ARTÍCULO 1°.- Acéptase, a partir del 10 de dic...",Renuncia,2019-12-10,29962252,"[2019-12-10 00:00:00, ARTÍCULO 1°.- Acéptase, ...",0,"{0: [2019-12-10 00:00:00, 'ARTÍCULO 1°.- Acépt..."
2249,SECRETARÍA DE GOBIERNO DE TRABAJO Y EMPLEO,Decreto 880/2019,2019-12-10,"ARTÍCULO 2°.- Acéptase, a partir del 10 de dic...",Renuncia,2019-12-10,25055401,"[2019-12-10 00:00:00, ARTÍCULO 2°.- Acéptase, ...",0,"{0: [2019-12-10 00:00:00, 'ARTÍCULO 2°.- Acépt..."
2250,SECRETARÍA DE GOBIERNO DE TRABAJO Y EMPLEO,Decreto 880/2019,2019-12-10,"ARTÍCULO 3°.- Acéptase, a partir del 10 de dic...",Renuncia,2019-12-10,25683272,"[2019-12-10 00:00:00, ARTÍCULO 3°.- Acéptase, ...",0,"{0: [2019-12-10 00:00:00, 'ARTÍCULO 3°.- Acépt..."
...,...,...,...,...,...,...,...,...,...,...
6906,MINISTERIO DE ECONOMÍA SECRETARÍA DE AGRICULTU...,Resolución 9/2024,2024-01-15,"ARTÍCULO 2°.- Dase por aceptada, a partir del ...",Renuncia,2024-01-15,14857388,"[2024-01-15 00:00:00, ARTÍCULO 2°.- Dase por a...",0,"{0: [2024-01-15 00:00:00, 'ARTÍCULO 2°.- Dase ..."
6917,MINISTERIO DE CAPITAL HUMANO SECRETARÍA DE EDU...,Resolución 4/2024,2024-01-16,"ARTÍCULO 1º.- Aceptar, a partir del 10 de ener...",Renuncia,2024-01-16,18299525,"[2024-01-16 00:00:00, ARTÍCULO 1º.- Aceptar, a...",0,"{0: [2024-01-16 00:00:00, 'ARTÍCULO 1º.- Acept..."
6939,MINISTERIO DE CAPITAL HUMANO,Decreto 68/2024,2024-01-23,"ARTÍCULO 1°.- Acéptase, a partir del 11 de ene...",Renuncia,2024-01-23,24270950,"[2024-01-23 00:00:00, ARTÍCULO 1°.- Acéptase, ...",0,"{0: [2024-01-23 00:00:00, 'ARTÍCULO 1°.- Acépt..."
6940,MINISTERIO DE CAPITAL HUMANO,Decreto 68/2024,2024-01-23,"ARTÍCULO 2°.- Acéptase, a partir del 11 de ene...",Renuncia,2024-01-23,23865082,"[2024-01-23 00:00:00, ARTÍCULO 2°.- Acéptase, ...",0,"{0: [2024-01-23 00:00:00, 'ARTÍCULO 2°.- Acépt..."


#Colapsamos nuestro Df en base a sus DNI. Unidad de Analisis pasa de ser "Resolución-Articulo" a "Funcionario"

In [52]:
df_collapsed = df.groupby('DNI Extraido').agg(list).reset_index()

In [53]:
df_collapsed

,DNI Extraido,Jurisdicción,Numero Resolución,Fecha de Resolución,Articulos,Clasificación_resolución,Fecha Más Cercana,identificacion,Designación_Flags,Nuevo_Diccionario
0,10003393,"[MINISTERIO DE SALUD, MINISTERIO DE SALUD, MIN...","[Decisión Administrativa 828/2020, Resolución ...","[2020-05-22 00:00:00, 2022-04-07 00:00:00, 202...",[ARTÍCULO 1º.- Dase por designada con carácter...,"[Designación, Prorroga, Prorroga, Prorroga]","[2020-02-01 00:00:00, 2022-04-07 00:00:00, 202...","[[2020-02-01 00:00:00, ARTÍCULO 1º.- Dase por ...","[1, 2, 2, 2]","[{1: [2020-02-01 00:00:00, 'ARTÍCULO 1º.- Dase..."
1,10017789,[MINISTERIO PÚBLICO],[Decreto 384/2020],[2020-04-21 00:00:00],"[ARTÍCULO 1º.- Dáse por aceptada, a partir del...",[Renuncia],[2020-03-19 00:00:00],"[[2020-03-19 00:00:00, ARTÍCULO 1º.- Dáse por ...",[0],"[{0: [2020-03-19 00:00:00, 'ARTÍCULO 1º.- Dáse..."
2,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[ARTÍCULO 1°.- Prorrógase, las designaciones t...","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{2: [2020-07-14 00:00:00, 'ARTÍCULO 1°.- Pror..."
3,10077705,[MINISTERIO DE ECONOMÍA],[Resolución 1613/2023],[2023-10-27 00:00:00],"[ARTÍCULO 1°.- Desígnase, a partir del dictado...",[Designación],[2022-03-22 00:00:00],"[[2022-03-22 00:00:00, ARTÍCULO 1°.- Desígnase...",[1],"[{1: [2022-03-22 00:00:00, 'ARTÍCULO 1°.- Desí..."
4,10125550,[MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITA...,"[Decisión Administrativa 1443/2020, Resolución...","[2020-08-11 00:00:00, 2021-04-19 00:00:00]",[ARTÍCULO 1°.- Dase por designada con carácter...,"[Designación, Prorroga]","[2020-07-01 00:00:00, 2021-04-19 00:00:00]","[[2020-07-01 00:00:00, ARTÍCULO 1°.- Dase por ...","[1, 2]","[{1: [2020-07-01 00:00:00, 'ARTÍCULO 1°.- Dase..."
...,...,...,...,...,...,...,...,...,...,...
3804,94228607,[MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS],[Decisión Administrativa 510/2023],[2023-06-26 00:00:00],[ARTÍCULO 2º.- Desígnase con carácter transito...,[Designación],[2023-06-26 00:00:00],"[[2023-06-26 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2023-06-26 00:00:00, 'ARTÍCULO 2º.- Desí..."
3805,94494582,[MINISTERIO DE DESARROLLO SOCIAL],[Decisión Administrativa 1001/2022],[2022-10-05 00:00:00],[ARTÍCULO 1º.- Dase por prorrogada la adscripc...,[Prorroga],[2022-10-05 00:00:00],"[[2022-10-05 00:00:00, ARTÍCULO 1º.- Dase por ...",[2],"[{2: [2022-10-05 00:00:00, 'ARTÍCULO 1º.- Dase..."
3806,94743066,[MINISTERIO DE SEGURIDAD],[Decisión Administrativa 296/2022],[2022-03-18 00:00:00],[ARTÍCULO 2º.- Desígnase con carácter transito...,[Designación],[2022-03-18 00:00:00],"[[2022-03-18 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2022-03-18 00:00:00, 'ARTÍCULO 2º.- Desí..."
3807,95806583,[MINISTERIO DE OBRAS PÚBLICAS],[Decisión Administrativa 453/2021],[2021-05-10 00:00:00],[ARTÍCULO 2°.- Desígnase con carácter transito...,[Designación],[2021-05-10 00:00:00],"[[2021-05-10 00:00:00, ARTÍCULO 2°.- Desígnase...",[1],"[{1: [2021-05-10 00:00:00, 'ARTÍCULO 2°.- Desí..."


#Eliminamos columna articulos ya que en este momento es innecesaria

In [54]:
df_collapsed.drop(columns='Articulos', inplace=True)

#Reconvertimos nuestra unidad de analisis a "Funcionario-Cargo"
Este codigo basicamente crea una nueva fila por funcionario por cada cargo que haya tenido este.

In [55]:
df = df_collapsed

# Función para dividir listas de eventos
def split_by_event_type(events):
    result = []
    temp = []
    for event in events:
        key = list(event.keys())[0]
        if key == 1:
            if temp:
                result.append(temp)
            temp = [event]
        else:
            temp.append(event)
    if temp:
        result.append(temp)
    return result

# Lista para almacenar las nuevas filas
new_rows = []

# Iterar sobre el DataFrame original
for _, row in df.iterrows():
    id_value = row['DNI Extraido']
    events_list = row['Nuevo_Diccionario']
    split_list = split_by_event_type(events_list)

    # Crear nuevas filas basadas en los resultados de split_by_event_type
    for sublist in split_list:
        new_row = row.to_dict()  # Copiar la fila actual como un diccionario
        new_row['Nuevo_Diccionario'] = sublist  # Actualizar la columna 'Eventos'
        new_rows.append(new_row)

# Crear un nuevo DataFrame con las nuevas filas
df_expanded = pd.DataFrame(new_rows)

df_expanded

,DNI Extraido,Jurisdicción,Numero Resolución,Fecha de Resolución,Clasificación_resolución,Fecha Más Cercana,identificacion,Designación_Flags,Nuevo_Diccionario
0,10003393,"[MINISTERIO DE SALUD, MINISTERIO DE SALUD, MIN...","[Decisión Administrativa 828/2020, Resolución ...","[2020-05-22 00:00:00, 2022-04-07 00:00:00, 202...","[Designación, Prorroga, Prorroga, Prorroga]","[2020-02-01 00:00:00, 2022-04-07 00:00:00, 202...","[[2020-02-01 00:00:00, ARTÍCULO 1º.- Dase por ...","[1, 2, 2, 2]","[{1: [2020-02-01 00:00:00, 'ARTÍCULO 1º.- Dase..."
1,10017789,[MINISTERIO PÚBLICO],[Decreto 384/2020],[2020-04-21 00:00:00],[Renuncia],[2020-03-19 00:00:00],"[[2020-03-19 00:00:00, ARTÍCULO 1º.- Dáse por ...",[0],"[{0: [2020-03-19 00:00:00, 'ARTÍCULO 1º.- Dáse..."
2,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{2: [2020-07-14 00:00:00, 'ARTÍCULO 1°.- Pror..."
3,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{1: [2020-10-14 00:00:00, 'ARTÍCULO 1º.- Dase..."
4,10077705,[MINISTERIO DE ECONOMÍA],[Resolución 1613/2023],[2023-10-27 00:00:00],[Designación],[2022-03-22 00:00:00],"[[2022-03-22 00:00:00, ARTÍCULO 1°.- Desígnase...",[1],"[{1: [2022-03-22 00:00:00, 'ARTÍCULO 1°.- Desí..."
...,...,...,...,...,...,...,...,...,...
4640,94228607,[MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS],[Decisión Administrativa 510/2023],[2023-06-26 00:00:00],[Designación],[2023-06-26 00:00:00],"[[2023-06-26 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2023-06-26 00:00:00, 'ARTÍCULO 2º.- Desí..."
4641,94494582,[MINISTERIO DE DESARROLLO SOCIAL],[Decisión Administrativa 1001/2022],[2022-10-05 00:00:00],[Prorroga],[2022-10-05 00:00:00],"[[2022-10-05 00:00:00, ARTÍCULO 1º.- Dase por ...",[2],"[{2: [2022-10-05 00:00:00, 'ARTÍCULO 1º.- Dase..."
4642,94743066,[MINISTERIO DE SEGURIDAD],[Decisión Administrativa 296/2022],[2022-03-18 00:00:00],[Designación],[2022-03-18 00:00:00],"[[2022-03-18 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2022-03-18 00:00:00, 'ARTÍCULO 2º.- Desí..."
4643,95806583,[MINISTERIO DE OBRAS PÚBLICAS],[Decisión Administrativa 453/2021],[2021-05-10 00:00:00],[Designación],[2021-05-10 00:00:00],"[[2021-05-10 00:00:00, ARTÍCULO 2°.- Desígnase...",[1],"[{1: [2021-05-10 00:00:00, 'ARTÍCULO 2°.- Desí..."


In [56]:
df_expanded

,DNI Extraido,Jurisdicción,Numero Resolución,Fecha de Resolución,Clasificación_resolución,Fecha Más Cercana,identificacion,Designación_Flags,Nuevo_Diccionario
0,10003393,"[MINISTERIO DE SALUD, MINISTERIO DE SALUD, MIN...","[Decisión Administrativa 828/2020, Resolución ...","[2020-05-22 00:00:00, 2022-04-07 00:00:00, 202...","[Designación, Prorroga, Prorroga, Prorroga]","[2020-02-01 00:00:00, 2022-04-07 00:00:00, 202...","[[2020-02-01 00:00:00, ARTÍCULO 1º.- Dase por ...","[1, 2, 2, 2]","[{1: [2020-02-01 00:00:00, 'ARTÍCULO 1º.- Dase..."
1,10017789,[MINISTERIO PÚBLICO],[Decreto 384/2020],[2020-04-21 00:00:00],[Renuncia],[2020-03-19 00:00:00],"[[2020-03-19 00:00:00, ARTÍCULO 1º.- Dáse por ...",[0],"[{0: [2020-03-19 00:00:00, 'ARTÍCULO 1º.- Dáse..."
2,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{2: [2020-07-14 00:00:00, 'ARTÍCULO 1°.- Pror..."
3,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{1: [2020-10-14 00:00:00, 'ARTÍCULO 1º.- Dase..."
4,10077705,[MINISTERIO DE ECONOMÍA],[Resolución 1613/2023],[2023-10-27 00:00:00],[Designación],[2022-03-22 00:00:00],"[[2022-03-22 00:00:00, ARTÍCULO 1°.- Desígnase...",[1],"[{1: [2022-03-22 00:00:00, 'ARTÍCULO 1°.- Desí..."
...,...,...,...,...,...,...,...,...,...
4640,94228607,[MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS],[Decisión Administrativa 510/2023],[2023-06-26 00:00:00],[Designación],[2023-06-26 00:00:00],"[[2023-06-26 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2023-06-26 00:00:00, 'ARTÍCULO 2º.- Desí..."
4641,94494582,[MINISTERIO DE DESARROLLO SOCIAL],[Decisión Administrativa 1001/2022],[2022-10-05 00:00:00],[Prorroga],[2022-10-05 00:00:00],"[[2022-10-05 00:00:00, ARTÍCULO 1º.- Dase por ...",[2],"[{2: [2022-10-05 00:00:00, 'ARTÍCULO 1º.- Dase..."
4642,94743066,[MINISTERIO DE SEGURIDAD],[Decisión Administrativa 296/2022],[2022-03-18 00:00:00],[Designación],[2022-03-18 00:00:00],"[[2022-03-18 00:00:00, ARTÍCULO 2º.- Desígnase...",[1],"[{1: [2022-03-18 00:00:00, 'ARTÍCULO 2º.- Desí..."
4643,95806583,[MINISTERIO DE OBRAS PÚBLICAS],[Decisión Administrativa 453/2021],[2021-05-10 00:00:00],[Designación],[2021-05-10 00:00:00],"[[2021-05-10 00:00:00, ARTÍCULO 2°.- Desígnase...",[1],"[{1: [2021-05-10 00:00:00, 'ARTÍCULO 2°.- Desí..."


In [57]:
df_expanded['Tiene_mas_de_un_evento'] = df_expanded['Nuevo_Diccionario'].apply(lambda x: 1 if len(x) > 1 else 0)

#Extraemos los valores del diccionario y los volvemos a meter como columnas individuales

In [58]:
def extract_values(dic_list):
    values = []
    for dic in dic_list:
        for key, value in dic.items():
            values.append(value)
    return values

# Crear una nueva columna con los valores extraídos
df_expanded['Valores_dict'] = df_expanded['Nuevo_Diccionario'].apply(extract_values)
lista_valores=df_expanded['Valores_dict'].tolist()

In [59]:
def extract_first_date(sublist): #Extraemos fecha de primer cargo designado
    if sublist and sublist[0]:
        return sublist[0][0]
    return None

def extract_ult_date(sublist):
    if sublist and sublist[0]:  #Extraemos ultima fecha registrada del funcionario en el cargo
        return sublist[-1][0]
    return None

def extract_primer_art(sublist):
    if sublist and sublist[0]:  #Extraemmos articulo por el cual el funcionario fue designado
        return sublist[0][1]
    return None


def extract_first_class(sublist):
    if sublist and sublist[0]:  #Vemos si la primer aparición del funcionario en el boletin oficial fue por una designación, una prorroga o una renuncia
        return sublist[0][-1]
    return None

def extract_last_class(sublist):
    if sublist and sublist[0]:  #Vemos si la ultima aparición del funcionario en el boletin oficial fue por una designación, una prorroga o una renuncia
        return sublist[-1][-1]
    return None

def extract_first_res(sublist):
    if sublist and sublist[0]:  #Extraemos el numero de la resolución por la que el funcionario aparecio por primera vez en el boletin oficial
        return sublist[0][2]
    return None

def extract_last_res(sublist):
    if sublist and sublist[0]:  #Extraemos el numero de la resolución por la que el funcionario aparecio por ultima vez en el boletin oficial
        return sublist[-1][2]
    return None

def extract_first_min(sublist):
    if sublist and sublist[0]:   #Extraemos el ministerio de la resolución por la que el funcionario aparecio por primera vez en el boletin oficial
        return sublist[0][3]
    return None

def extract_last_min(sublist):
    if sublist and sublist[0]:  #Extraemos el ministerio de la resolución por la que el funcionario aparecio por ultima vez en el boletin oficial
        return sublist[-1][3]
    return None


# Crear una nueva columna con la fecha del primer elemento de cada sublista
df_expanded['first_art'] = df_expanded['Valores_dict'].apply(extract_primer_art)
df_expanded['first_date'] = df_expanded['Valores_dict'].apply(extract_first_date)
df_expanded['ult_date'] = df_expanded['Valores_dict'].apply(extract_ult_date)
df_expanded['first_class'] = df_expanded['Valores_dict'].apply(extract_first_class)
df_expanded['last_class'] = df_expanded['Valores_dict'].apply(extract_last_class)
df_expanded['first_res'] = df_expanded['Valores_dict'].apply(extract_first_res)
df_expanded['last_res'] = df_expanded['Valores_dict'].apply(extract_last_res)
df_expanded['first_min'] = df_expanded['Valores_dict'].apply(extract_first_min)
df_expanded['last_min'] = df_expanded['Valores_dict'].apply(extract_last_min)

In [60]:
df_expanded = df_expanded[df_expanded['first_class'] == "Designación"] #Nos quedamos solo con los funcionarios cuya primera aparición haya sido en una designación

In [61]:
df_expanded

,DNI Extraido,Jurisdicción,Numero Resolución,Fecha de Resolución,Clasificación_resolución,Fecha Más Cercana,identificacion,Designación_Flags,Nuevo_Diccionario,Tiene_mas_de_un_evento,Valores_dict,first_art,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,last_min
0,10003393,"[MINISTERIO DE SALUD, MINISTERIO DE SALUD, MIN...","[Decisión Administrativa 828/2020, Resolución ...","[2020-05-22 00:00:00, 2022-04-07 00:00:00, 202...","[Designación, Prorroga, Prorroga, Prorroga]","[2020-02-01 00:00:00, 2022-04-07 00:00:00, 202...","[[2020-02-01 00:00:00, ARTÍCULO 1º.- Dase por ...","[1, 2, 2, 2]","[{1: [2020-02-01 00:00:00, 'ARTÍCULO 1º.- Dase...",1,"[[2020-02-01 00:00:00, ARTÍCULO 1º.- Dase por ...",ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,MINISTERIO DE SALUD
3,10068119,"[JEFATURA DE GABINETE DE MINISTROS, JEFATURA D...","[Resolución 291/2020, Decisión Administrativa ...","[2020-07-14 00:00:00, 2021-07-06 00:00:00]","[Prorroga, Designación]","[2020-07-14 00:00:00, 2020-10-14 00:00:00]","[[2020-07-14 00:00:00, ARTÍCULO 1°.- Prorrógas...","[2, 1]","[{1: [2020-10-14 00:00:00, 'ARTÍCULO 1º.- Dase...",0,"[[2020-10-14 00:00:00, ARTÍCULO 1º.- Dase por ...",ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS
4,10077705,[MINISTERIO DE ECONOMÍA],[Resolución 1613/2023],[2023-10-27 00:00:00],[Designación],[2022-03-22 00:00:00],"[[2022-03-22 00:00:00, ARTÍCULO 1°.- Desígnase...",[1],"[{1: [2022-03-22 00:00:00, 'ARTÍCULO 1°.- Desí...",0,"[[2022-03-22 00:00:00, ARTÍCULO 1°.- Desígnase...","ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,MINISTERIO DE ECONOMÍA
5,10125550,[MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITA...,"[Decisión Administrativa 1443/2020, Resolución...","[2020-08-11 00:00:00, 2021-04-19 00:00:00]","[Designación, Prorroga]","[2020-07-01 00:00:00, 2021-04-19 00:00:00]","[[2020-07-01 00:00:00, ARTÍCULO 1°.- Dase por ...","[1, 2]","[{1: [2020-07-01 00:00:00, 'ARTÍCULO 1°.- Dase...",1,"[[2020-07-01 00:00:00, ARTÍCULO 1°.- Dase por ...",ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT
6,10134579,[JEFATURA DE GABINETE DE MINISTROS],[Decisión Administrativa 1451/2020],[2020-08-13 00:00:00],[Designación],[2020-08-13 00:00:00],"[[2020-08-13 00:00:00, ARTÍCULO 1°.- Desígnase...",[1],"[{1: [2020-08-13 00:00:00, 'ARTÍCULO 1°.- Desí...",0,"[[2020-08-13 00:00:00, ARTÍCULO 1°.- Desígnase...","ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,94126199,"[MINISTERIO DE TRANSPORTE, MINISTERIO DE TRANS...","[Decisión Administrativa 755/2022, Resolución ...","[2022-08-01 00:00:00, 2023-06-07 00:00:00]","[Designación, Prorroga]","[2022-08-01 00:00:00, 2023-06-07 00:00:00]","[[2022-08-01 00:00:00, ARTÍCULO 2°.- Desígnase...","[1, 2]","[{1: [2022-08-01 00:00:00, 'ARTÍCULO 2°.- Desí...",1,"[[2022-08-01 00:00:00, ARTÍCULO 2°.- Desígnase...",ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,MINISTERIO DE TRANSPORTE
4640,94228607,[MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS],[Decisión Administrativa 510/2023],[2023-06-26 00:00:00],[Designación],[2023-06-26 

#Utilizando DATUAR para scrapear nombres, domicilio y edad

Esta es la función que use para extraer los datos de Datuar. No obstante, parece que no funcionaria nuevamente a raiz de que Datuar ha cambiado la estructura de su pagina, volviendose mas que nada un medio pago. Lo unico que podriamos extraer libremente de Datuar ahora seria el nombre de los funcionarios. En su momento, use Selenium para interactuar con los elementos javascript de la pagina, ejecutar busquedas por cada DNI extraido, y extraer los datos de cada funcionario.

In [62]:
# Ultima actualizacion / marzo 2024 (a Julio 2024 datuar cambio la estructura, cuidado cuando ejecutas el codigo)
# !pip install selenium
# !apt-get update
# !apt-get install -y chromium-browser
# !apt-get install chromium-chromedriver
# !ls /usr/lib/chromium-browser/chromedriver

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys

# from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time
# from selenium.common.exceptions import TimeoutException

# options = webdriver.ChromeOptions()
# options.add_argument("--no-sandbox")
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(options=options)

# numeros_extraidos=df_filtrado_designaciones["DNI Extraido"].tolist()
# partes = 4
# sublistas = dividir_lista(numeros_extraidos, partes)

# url = "https://datuar.com/"
# numeros_a_buscar = sublistas[4]
# lista_resultados = []
# max_intentos = 5
# for numero in numeros_a_buscar:
#     resultado_actual = {"Numero": numero, "Resultados": []}
#     intentos = 0

#     while intentos < max_intentos:
#         try:
#             driver.get(url)

#             barra_de_busqueda = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, "criterio"))
#             )

#             barra_de_busqueda.clear()
#             barra_de_busqueda.send_keys(numero)
#             barra_de_busqueda.send_keys(Keys.RETURN)
#             resultados = WebDriverWait(driver, 10).until(
#                 EC.presence_of_all_elements_located((By.CLASS_NAME, "txt_royal_blue"))
#             )

#             if len(resultados) > 0:
#                 for resultado in resultados:
#                     resultado_actual["Resultados"].append(resultado.text)
#             else:
#                 resultado_actual["Resultados"].append("No se encontraron resultados para este número")

#             break  # Salir del bucle si no hay TimeoutException

#         except TimeoutException:
#             intentos += 1
#             print(f"Intento {intentos} - No se pudo encontrar resultados para el número: {numero}")

#     if intentos == max_intentos:
#         print(f"Se agotaron los intentos para el número: {numero}")
#         resultado_actual["Resultados"].append("Error de tiempo de espera")
#     lista_resultados.append(resultado_actual)

# df_resultados = pd.DataFrame(lista_resultados)
# df_resultados
# df_resultados.to_excel("Datuar_perdidos_4.xlsx", index=False)
# files.download('Datuar_perdidos_4.xlsx')

In [63]:
#Importamos los dataframes de Datuar que cree a traves del scrapeo
def importar_y_asignar_nombres(nombres_archivos):
    dfs = {}
    for nombre_archivo in nombres_archivos:
        try:
            nombre_df = nombre_archivo.split('.')[0]
            dfs[nombre_df] = pd.read_excel(nombre_archivo)
        except Exception as e:
            print(f"Error al importar el archivo {nombre_archivo}: {e}")
    return dfs
nombres_archivos = [
    "Datuar_perdidos_0.xlsx",
    "Datuar_perdidos_1.xlsx",
    "Datuar_perdidos_2.xlsx",
    "Datuar_perdidos_3.xlsx",
    "Datuar_perdidos_4.xlsx",
    "Datuar_perdidos_4(1).xlsx",
    "Datuar_provincias_0.xlsx",
    "Datuar_provincias_1.xlsx",
    "Datuar_provincias_1 (1).xlsx",
    "Datuar_provincias_3.xlsx",
    "Data_dni_Amb_0.xlsx",
    "Data_dni_Cult_0.xlsx",
    "Data_dni_D_Soc_0.xlsx",
    "Data_dni_D_Terr_0.xlsx",
    "Data_dni_Def_0.xlsx",
    "Data_dni_eco_0.xlsx",
    "Data_dni_eco_completo(1).xlsx",
    "Data_dni_eco.xlsx",
    "Data_dni_Edu_0.xlsx",
    "Data_dni_Just_0.xlsx",
    "Data_dni_Mujer_0.xlsx",
    "Data_dni_Obras_0.xlsx",
    "Data_dni_Sal_0.xlsx",
    "Data_dni_Seg_0.xlsx",
    "Data_dni_Trab_0.xlsx"
]
dataframes = importar_y_asignar_nombres(nombres_archivos)
df_unificado_datuar = pd.concat(dataframes.values(), ignore_index=True)

In [64]:
#Creamos una lista de nombres. No obstante, esta lista contendra elementos que no son de interes, como el cuil, ya que estan en la misma columna.
lista_nombres_datuar=df_unificado_datuar["Resultados"].tolist()

In [65]:
#Esta función sirve para quedarnos solo con los nombres.
datos = lista_nombres_datuar
datos_separados = [elemento.split(", '") for elemento in datos]
longitud_maxima = max(len(sublista) for sublista in datos_separados)
datos_sin_vacios = [[elemento for elemento in sublista if elemento != ""] + [''] * (longitud_maxima - len(sublista)) for sublista in datos_separados]
listas_elementos = [[] for _ in range(longitud_maxima)]

for sublista in datos_sin_vacios:
    for i, elemento in enumerate(sublista):
        listas_elementos[i].append(elemento.strip("'"))

print(listas_elementos[0])

["['DEMEY, RODRIGO MARTIN", "['OLGUIN, DIANA GABRIELA", "['FREIDEMBERG, ANABELLA", "['ALMIRON DENIS, EDUARDO IGNACIO", "['OSORIO, HECTOR LAUREANO", "['MAZA, DIEGO JAVIER", "['PEREZ, MARIA FLORENCIA", "['LICARI, LISANDRO RAMON", "['GIGANTE, EDUARDO JOSE", "['IRACHET, IGNACIO", "['GOMEZ, NANCY VALERIA", "['PALAZZO, ALBERTO GABRIEL", "['GRUNBLATT, RUTH SALOMÉ", "['BERMUDEZ, ANDREA RAQUEL", "['ORDUNA, HERNÁN DARÍO", "['CALDERON, MARIA PAULINA", "['BRAGA BEATOVE, MARIA CELESTE", "['VACA NARVAJA , PATRICIA", "['AMBROSI, DANIELA CELESTE", "['GARAY, CECILIA GABRIELA", "['BARUTTA, MATÍAS EZEQUIEL", "['HENRIQUES DE BARROS, GERMAN NICOLAS", "['LOMBARDI, SANDRA ANDREA", "['PARIS, RAUL EDUARDO", "['AGÜERA, WALTER ALBERTO", "['MELARAÑA, EZEQUIEL CARLOS", "['BLANCO MOLLO, FRANCO DANIEL", "['ALMECIJA, JULIETA", "['FASKOWICZ, GUSTAVO ADRIAN", "['ROJAS, NESTOR ALBERTO", "['MINERVINI, OSVALDO EUGENIO", "['OSPITAL, CARLOS ROBERTO", "['DOMINGUEZ, DIEGO FERNANDO", "['FARIAS, MARINA DANIELA", "['SOSA, NAHUEL

In [66]:
#Esta función sirve para transformar los nombres a su formato adecuado (de "APELLIDO, NOMBRE" a "Nombre Apellido")
lista_nombres = listas_elementos[0]
lista_nombres_formateados = []
for nombre_completo in lista_nombres:
    if ", " in nombre_completo:
        nombre_completo = nombre_completo.replace("[", "").replace("'", "")
        apellido, nombre = nombre_completo.split(", ")
        nombre_completo_formateado = (nombre + " " + apellido).title()
        lista_nombres_formateados.append(nombre_completo_formateado)
    else:
        lista_nombres_formateados.append(None)
print(lista_nombres_formateados)

['Rodrigo Martin Demey', 'Diana Gabriela Olguin', 'Anabella Freidemberg', 'Eduardo Ignacio Almiron Denis', 'Hector Laureano Osorio', 'Diego Javier Maza', 'Maria Florencia Perez', 'Lisandro Ramon Licari', 'Eduardo Jose Gigante', 'Ignacio Irachet', 'Nancy Valeria Gomez', 'Alberto Gabriel Palazzo', 'Ruth Salomé Grunblatt', 'Andrea Raquel Bermudez', 'Hernán Darío Orduna', 'Maria Paulina Calderon', 'Maria Celeste Braga Beatove', 'Patricia Vaca Narvaja ', 'Daniela Celeste Ambrosi', 'Cecilia Gabriela Garay', 'Matías Ezequiel Barutta', 'German Nicolas Henriques De Barros', 'Sandra Andrea Lombardi', 'Raul Eduardo Paris', 'Walter Alberto Agüera', 'Ezequiel Carlos Melaraña', 'Franco Daniel Blanco Mollo', 'Julieta Almecija', 'Gustavo Adrian Faskowicz', 'Nestor Alberto Rojas', 'Osvaldo Eugenio Minervini', 'Carlos Roberto Ospital', 'Diego Fernando Dominguez', 'Marina Daniela Farias', 'Nahuel Sosa', 'Yesica Mayra Santacruz', 'Maria Gabriela Labiano', 'Francisco Abramovich', 'Agustin Omar Andres', 'Ju

In [67]:
#Extraemos de la lista de nombres original los otros datos de interes (cuit, edad, provincia y ciudad), y los metemos en sus propias listas

#Lista de CUIT
lista_cuits = listas_elementos[1]
lista_cuits_perdidos = [dato.split(": ")[1] if ":" in dato else dato for dato in lista_cuits]

#Lista de edades
lista_edades = listas_elementos[2]
lista_edades_perdidos = [dato.split(": ")[1] if ":" in dato else dato for dato in lista_edades]

#Lista de provincias
lista_provincias = listas_elementos[3]
lista_provincias_perdidos = [dato.split(": ")[1] if ":" in dato else dato for dato in lista_provincias]

#Lista de ciudades
lista_ciudades = listas_elementos[4]
lista_ciudades_perdidos = [dato.split(": ")[1] if ":" in dato else dato for dato in lista_ciudades]

print("CUIT:", lista_cuits_perdidos)
print("Edades:", lista_edades_perdidos)
print("Provincias:", lista_provincias_perdidos)
print("Ciudades:", lista_ciudades_perdidos)

CUIT: ['20-38070271-2', '27-31675270-0', '27-33464400-1', '20-32439480-0', '20-17752060-9', '20-24589350-8', '27-36929512-3', '20-32874713-9', '20-23515299-2', '20-33157309-5', '27-21474243-3', '20-29646725-2', '27-30226273-5', '27-22709885-1', '20-07992633-8', '27-26042710-0', '27-28697820-2', '27-11747000-3', '27-31369844-6', '27-29508972-0', '20-25654393-2', '20-35881409-4', '23-20477342-4', '20-30597000-0', '20-28174440-3', '20-23284517-2', '20-30460400-0', '27-25129851-9', '20-22147412-1', '20-10500325-1', '20-13059610-0', '20-13313970-3', '20-31227992-5', '27-27060760-3', '20-32480953-9', '27-34253396-0', '27-21478155-2', '20-33545278-0', '20-25710324-3', '20-32668897-6', '20-22990377-3', '27-34828509-8', '27-22818077-2', '20-31453707-7', '27-27528020-3', '27-32843923-4', '20-27279944-0', '27-34598970-1', '20-24492710-7', '23-29067241-4', '20-14729480-9', '27-14439500-5', '27-20860120-8', '20-32748683-8', '27-28693940-1', '27-12601341-3', '20-20568520-1', '27-33976517-6', '27-249

In [68]:
#Eliminamos el texto que esta dentro de cada elemento de la lista de edades, para quedarnos unicamente con los numeros
import re
lista_edades = lista_edades_perdidos
def extraer_numeros(cadena):
    numeros_encontrados = re.findall(r'\d+', cadena)
    if numeros_encontrados:
        return int(numeros_encontrados[0])
    else:
        return "No encontrado"
lista_edades_numeros = [extraer_numeros(edad) for edad in lista_edades]
print(lista_edades_numeros)

[30, 38, 36, 37, 57, 48, 31, 37, 50, 36, 54, 41, 40, 51, 77, 46, 43, 68, 39, 41, 45, 31, 55, 40, 43, 51, 40, 48, 52, 71, 65, 66, 45, 45, 37, 35, 54, 36, 47, 37, 51, 34, 51, 39, 44, 37, 45, 34, 48, 42, 61, 63, 54, 37, 43, 65, 55, 35, 48, 64, 28, 39, 46, 48, 29, 48, 63, 20, 40, 65, 37, 48, 35, 41, 44, 32, 24, 51, 30, 44, 45, 49, 40, 52, 44, 37, 37, 66, 30, 26, 52, 39, 34, 38, 38, 33, 39, 70, 66, 44, 47, 37, 44, 45, 59, 43, 30, 38, 40, 49, 34, 52, 58, 30, 39, 34, 52, 34, 64, 39, 55, 32, 44, 44, 38, 61, 64, 27, 41, 50, 41, 60, 47, 31, 47, 39, 41, 37, 47, 40, 63, 62, 21, 37, 44, 45, 50, 44, 31, 27, 57, 43, 33, 69, 33, 51, 26, 29, 30, 37, 38, 39, 38, 45, 56, 51, 58, 50, 57, 46, 42, 46, 47, 43, 29, 30, 37, 34, 51, 51, 37, 37, 39, 42, 35, 35, 59, 50, 54, 33, 37, 64, 63, 50, 61, 26, 55, 53, 33, 29, 35, 39, 45, 43, 52, 38, 50, 54, 33, 69, 44, 45, 59, 35, 33, 47, 63, 47, 40, 50, 39, 27, 33, 61, 40, 31, 72, 33, 64, 42, 43, 61, 36, 66, 49, 33, 48, 36, 37, 36, 40, 52, 49, 40, 49, 54, 47, 57, 55, 54,

In [69]:
lista_DNI_datuar=df_unificado_datuar["Numero"].tolist() #Creamos una lista con los DNI del df de datuar

In [70]:
#Creamos un nuevo df con los datos "limpios"
df_dnis_perdidos=pd.DataFrame({"DNI": lista_DNI_datuar, "Nombre Completo": lista_nombres_formateados, "CUIT": lista_cuits_perdidos, "Edad": lista_edades_numeros, "Provincia": lista_provincias_perdidos, "Ciudad": lista_ciudades_perdidos})

In [71]:
df_dnis_perdidos

,DNI,Nombre Completo,CUIT,Edad,Provincia,Ciudad
0,38070271,Rodrigo Martin Demey,20-38070271-2,30,Ciudad De Buenos Aires,Núñez
1,31675270,Diana Gabriela Olguin,27-31675270-0,38,Ciudad De Buenos Aires,Caballito
2,33464400,Anabella Freidemberg,27-33464400-1,36,Buenos Aires,Berazategui
3,32439480,Eduardo Ignacio Almiron Denis,20-32439480-0,37,Ciudad De Buenos Aires,Palermo
4,17752060,Hector Laureano Osorio,20-17752060-9,57,Buenos Aires,González Catán
...,...,...,...,...,...,...
4795,24143386,Verónica Derna,27-24143386-8,49,Misiones,Posadas
4796,5266107,Victor Hugo Hissa,20-05266107-3,75,San Luis,El Trapiche
4797,22672846,Walter Marcelo Santillan,20-22672846-6,51,Tucuman,Bella Vista
4798,24872128,Ximena Guadalupe Merlo Avila,27-24872128-1,48,Ciudad De Buenos Aires,Balvanera


In [72]:
df_dnis_perdidos = df_dnis_perdidos.dropna(subset=['Nombre Completo']) #Eliminamos las observaciones que tengan asignado "Nan" en la columna nombre completo

In [73]:
lista_dnis_perdidos_string=df_dnis_perdidos["DNI"].tolist()
lista_dnis_perdidos_string = [str(elemento) for elemento in lista_dnis_perdidos_string]

#Importante: Nombres que se añadieron manualmente
Los nombres que no fueron registrados por datuar y se debieron añadir manualmente a este df son:
Carlos Alberto PAZ,
Nicolás GÓMEZ,
Alicia Guadalupe CEJAS,
Héctor Alan DOTTORE,
Juan Fernando CABAÑA,
Silvia Beatriz PEREZ,
Carlos Miguel MILICEVIC,
Gabriela Delia FERNÁNDEZ,
Agustín Marcos ROMERO,
Gerardo Abel DÍAZ BARTOLOMÉ,
Javier Humberto TABORDA,
Sergio Tomás MASSA,
Marcos Daniel VAGO,
Silvina Verónica FEDIUK,
Adriana Isabel AVACA,
Romina WUVCZARYK,
Fernando Ernesto PEIRANO,
Natalia Elizabeth GERULA,
Mariana Eva TELLO,
Martin Omar EL TAHHAM,
Laura ROMANO,
Mariana GAMBERA,
Verónica FIORITO,
Facundo Javier GALVANI,
Marcos Adolfo SAUCEDO,
Samanta Ivana SILVA,
Cecilia Verónica BRUSCA,
Natalia Anahí RUBIO,
Pablo Bernardo ARBURÚA,
Pedro Atilio FERNANDEZ,
Julieta REY,
Susana Pilar ALDERETE GUEVARA,
Carlos Alberto PAZ,
Nicolás GÓMEZ,
Alicia Guadalupe CEJAS,
Héctor Alan DOTTORE,
Juan Fernando CABAÑA,
Silvia Beatriz PEREZ,
Carlos Miguel MILICEVIC,
Gabriela Delia FERNÁNDEZ,
Agustín Marcos ROMERO,
Gerardo Abel DÍAZ BARTOLOMÉ,
Javier Humberto TABORDA,
Sergio Tomás MASSA,
Marcos Daniel VAGO,
Silvina Verónica FEDIUK,
Adriana Isabel AVACA,
Romina WUVCZARYK,
Fernando Ernesto PEIRANO,
Natalia Elizabeth GERULA,
Mariana Eva TELLO,
Martin Omar EL TAHHAM,
Laura ROMANO,
Mariana GAMBERA,
Verónica FIORITO,
Facundo Javier GALVANI,
Marcos Adolfo SAUCEDO,
Samanta Ivana SILVA,
Cecilia Verónica BRUSCA,
Natalia Anahí RUBIO,
Pablo Bernardo ARBURÚA,
Pedro Atilio FERNANDEZ,
Julieta REY,
Susana Pilar ALDERETE GUEVARA

Los nombres que estaban mal registrados en Datuar y que se debieron arreglar manualmente son:
Eduardo Horacio Eugenio YAZBECK JOZAMI, Eduardo Salvador BARCESAT, Victorio María José TACCETTI, Miguel Ángel MAZZA CAMPOS, Aron Mirena Izaguirre, Luis Juan OLMO, Alfredo Juan FALÚ, Hilda Isolina DOMÍNGUEZ, Miguel Ángel BAELO, Daniel MARADEI, Liliana Noemí PASCUAL, Pablo Antonio FONTDEVILA, Jorge Enrique TAIANA


In [ ]:
data=pd.read_excel("df_dnis_perdidos_2.xlsx")
df_dnis_perdidos=pd.DataFrame(data)

#Mapeando datos de funcionarios

In [74]:
#Mapeamos nombre de funcionarios
diccionario_dnis_perdidos = dict(zip(df_dnis_perdidos['DNI'], df_dnis_perdidos['Nombre Completo']))
diccionario_dnis_perdidos = {str(key): value for key, value in diccionario_dnis_perdidos.items()}
df_expanded['Nombre Completo'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_dnis_perdidos)

diccionario_perdidos = dict(zip(df_dnis_perdidos['DNI'], df_dnis_perdidos['CUIT']))
diccionario_perdidos = {str(key): value for key, value in diccionario_perdidos.items()}
df_expanded['CUIT'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_perdidos)

diccionario_perdidos = dict(zip(df_dnis_perdidos['DNI'], df_dnis_perdidos['Edad']))
diccionario_perdidos = {str(key): value for key, value in diccionario_perdidos.items()}
df_expanded['Edad'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_perdidos)

diccionario_perdidos = dict(zip(df_dnis_perdidos['DNI'], df_dnis_perdidos['Provincia']))
diccionario_perdidos = {str(key): value for key, value in diccionario_perdidos.items()}
df_expanded['Provincia'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_perdidos)

diccionario_perdidos = dict(zip(df_dnis_perdidos['DNI'], df_dnis_perdidos['Ciudad']))
diccionario_perdidos = {str(key): value for key, value in diccionario_perdidos.items()}
df_expanded['Ciudad'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_perdidos)

<ipython-input-74-26baabab0480>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expanded['Nombre Completo'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_dnis_perdidos)
<ipython-input-74-26baabab0480>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expanded['CUIT'] = df_expanded['DNI Extraido'].astype(str).map(diccionario_perdidos)
<ipython-input-74-26baabab0480>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [75]:
df_expanded_filtrado=df_expanded[df_expanded["Nombre Completo"].notna()]

In [76]:
df_expanded_filtrado_2 = df_expanded_filtrado.drop(columns=['Jurisdicción', 'Numero Resolución',
       'Fecha de Resolución', 'Clasificación_resolución', 'Fecha Más Cercana',
       'identificacion', 'Designación_Flags', 'Nuevo_Diccionario', 'Valores_dict'])

#Agregando via excepción, transitoriedad y planta permanente

In [77]:
df_expanded_filtrado_2['Via Excepción'] = False
df_expanded_filtrado_2.loc[df_expanded_filtrado_2['first_art'].str.contains('con carácter de excepción|autorización excepcional', case=False), 'Via Excepción'] = True

In [78]:
df_expanded_filtrado_2['Designación Transitoria'] = False
df_expanded_filtrado_2.loc[df_expanded_filtrado_2['first_art'].str.contains('transitoriamente|carácter transitorio|caracter transitorio', case=False), 'Designación Transitoria'] = True

In [79]:
df_expanded_filtrado_2['Planta Permanente'] = False
df_expanded_filtrado_2.loc[df_expanded_filtrado_2['first_art'].str.contains('planta permanente|Planta Permanente', case=False), 'Planta Permanente'] = True

#Agregando ultimas fechas de funcionarios en cargos

In [80]:
import numpy as np
df_expanded_filtrado_2['ult_date'] = pd.to_datetime(df_expanded_filtrado_2['ult_date'])

# Crear la columna C con las condiciones especificadas usando apply
df_expanded_filtrado_2['Venc_Desig_prorr'] = df_expanded_filtrado_2.apply(lambda row: row['ult_date'] + pd.DateOffset(days=180) if row['last_class'] == 'Prorroga' else np.nan, axis=1)

In [81]:
df_expanded_filtrado_2['ult_date'] = pd.to_datetime(df_expanded_filtrado_2['ult_date'])

# Crear la columna C con las condiciones especificadas usando apply
df_expanded_filtrado_2['C'] = df_expanded_filtrado_2.apply(lambda row: row['ult_date'] + pd.DateOffset(days=180) if row['last_class'] == 'Designación' and row['Designación Transitoria'] else np.nan, axis=1)

df_expanded_filtrado_2['Venc_Desig_prorr'] = df_expanded_filtrado_2['Venc_Desig_prorr'].fillna(df_expanded_filtrado_2['C'])

df_expanded_filtrado_2=df_expanded_filtrado_2.drop(columns=['C'])

In [82]:
df_expanded_filtrado_2

,DNI Extraido,Tiene_mas_de_un_evento,first_art,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,last_min,Nombre Completo,CUIT,Edad,Provincia,Ciudad,Via Excepción,Designación Transitoria,Planta Permanente,Venc_Desig_prorr
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,MINISTERIO DE SALUD,Josefina Magdalena Meza,27-10003393-9,72.0,Formosa,Formosa,True,True,False,2024-03-30
3,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS,Ricardo Luis Teran,20-10068119-7,71.0,Santa Fe,No Encontrado,True,True,False,2021-04-12
4,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,MINISTERIO DE ECONOMÍA,Oscar Horacio Padin,20-10077705-4,72.0,Ciudad De Buenos Aires,Palermo,False,False,True,NaT
5,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Elsa Liliana Garcia,27-10125550-1,72.0,Ciudad De Buenos Aires,Caballito,True,True,False,2021-10-16
6,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS,Victoria Lichtschein,27-10134579-9,72.0,Ciudad De Buenos Aires,Palermo,True,True,True,2021-02-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,MINISTERIO DE TRANSPORTE,Maria Elisa Osano Cerdeña,27-94126199-5,36.0,Ciudad De Buenos Aires,Villa Crespo,True,True,False,2023-12-04
4640,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,Sergio Matias Yahari Vazquez,20-94228607-5,33.0,Buenos Aires,Paso Del Rey,True,True,False,2023-12-23
4642,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,MINISTERIO DE SEGURIDAD,Teresa De Jesus Colman Zarate,27-94743066-7,29.0,Ciudad De Buenos Aires,No Encontrado,True,True,False,2022-09-14
4643,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,MINISTERIO DE OBRAS PÚBLICAS,Renee Carolina Cordova Malaver,27-95806583-9,44.0,No Encontrado,No Encontrado,True,True,False,2021-11-06


#Agregando nivel y grado

In [83]:
#Función que busca grado y nivel del funcionario en base a los dos patrones existentes en el boletin oficial, y que despues los transforma a un unico patron
def buscar_grado_nivel(texto):
    patron_original = r'Nivel ([A-Z])\s*-?\s*Grado (\d{1,3})'
    coincidencias = re.findall(patron_original, texto)
    if coincidencias:
        nivel, grado = coincidencias[0]
        return f"Nivel {nivel} - Grado {grado}"

    patron_alternativo = r'Nivel ([A-Z]), Grado (\d{1,3})'
    coincidencias_alternativas = re.findall(patron_alternativo, texto)
    if coincidencias_alternativas:
        nivel, grado = coincidencias_alternativas[0]
        return f"Nivel {nivel} - Grado {grado}"

    patron_actualizado = r'Nivel ([A-Z])[\s–-]+Grado (\d{1,3})'
    coincidencias_alternativas = re.findall(patron_actualizado, texto)
    if coincidencias_alternativas:
        nivel, grado = coincidencias_alternativas[0]
        return f"Nivel {nivel} - Grado {grado}"

    patron_actualizado_2 = r'Nivel ([A-Z])\s*-?\s*Grado (\d{1,3})'
    coincidencias_alternativas = re.findall(patron_actualizado_2, texto)
    if coincidencias_alternativas:
        nivel, grado = coincidencias_alternativas[0]
        return f"Nivel {nivel} - Grado {grado}"
    return None
df_expanded_filtrado_2['Grado y Nivel'] = df_expanded_filtrado_2['first_art'].apply(buscar_grado_nivel)

In [84]:
df_expanded_filtrado_2

,DNI Extraido,Tiene_mas_de_un_evento,first_art,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Nombre Completo,CUIT,Edad,Provincia,Ciudad,Via Excepción,Designación Transitoria,Planta Permanente,Venc_Desig_prorr,Grado y Nivel
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,Josefina Magdalena Meza,27-10003393-9,72.0,Formosa,Formosa,True,True,False,2024-03-30,Nivel B - Grado 0
3,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,Ricardo Luis Teran,20-10068119-7,71.0,Santa Fe,No Encontrado,True,True,False,2021-04-12,Nivel B - Grado 0
4,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,Oscar Horacio Padin,20-10077705-4,72.0,Ciudad De Buenos Aires,Palermo,False,False,True,NaT,Nivel A - Grado 8
5,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,Elsa Liliana Garcia,27-10125550-1,72.0,Ciudad De Buenos Aires,Caballito,True,True,False,2021-10-16,Nivel B - Grado 0
6,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,Victoria Lichtschein,27-10134579-9,72.0,Ciudad De Buenos Aires,Palermo,True,True,True,2021-02-09,Nivel B - Grado 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,Maria Elisa Osano Cerdeña,27-94126199-5,36.0,Ciudad De Buenos Aires,Villa Crespo,True,True,False,2023-12-04,Nivel B - Grado 0
4640,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,Sergio Matias Yahari Vazquez,20-94228607-5,33.0,Buenos Aires,Paso Del Rey,True,True,False,2023-12-23,Nivel C - Grado 0
4642,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,Teresa De Jesus Colman Zarate,27-94743066-7,29.0,Ciudad De Buenos Aires,No Encontrado,True,True,False,2022-09-14,Nivel C - Grado 0
4643,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,Renee Carolina Cordova Malaver,27-95806583-9,44.0,No Encontrado,No Encontrado,True,True,False,2021-11-06,Nivel A - Grado 0


In [85]:
df_filtrado_designaciones=df_expanded_filtrado_2
df_filtrado_designaciones = df_filtrado_designaciones.rename(columns={'first_art': 'Parrafos_filtrados'})

#Cargo [NO CORRER]

In [ ]:
#Función que extrae las palabrbas que se encuentran entre ciertas frases clave, las cuales tienden a ser los cargos objetivo. No obstante, las combinaciones suelen ser
#casi infinitas.
import numpy as np

def extraer_texto_entre_palabras(parrafos):
    texto_extraido = []
    for parrafo in parrafos:
        if pd.isna(parrafo) or not parrafo:
            texto_extraido.append(np.nan)
            continue
        conjuntos_palabras = [
            ("cargo de", "de la SECRETARÍA"),
            ("cargo de", "de la entonces"),
            ("cargo de", "del MINISTERIO"),
            ("cargo de", "de la SUBSECRETARÍA"),
            ("cargo de", "de la DIRECCIÓN"),
            ("cargo de", "de la JEFATURA"),
            ("cargo de", "del INSTITUTO"),
            ("cargo de", "de la FUNDACIÓN"),
            ("cargo de", "de la Fundación"),
            ("cargo de", "de la fundación"),
            ("cargo de", "de la COMISIÓN"),
            ("cargo de", "de la Comisión"),
            ("cargo de", "de la comisión"),
            ("cargo de", "de la secretaría"),
            ("cargo de", "del ministerio"),
            ("cargo de", "de la subsecretaría"),
            ("cargo de", "de la dirección"),
            ("cargo de", "de la DIRECCIÓN"),
            ("cargo de", "de la jefatura"),
            ("cargo de", "del instituto"),
            ("cargo de", "de la Secretaría"),
            ("cargo de", "del Ministerio"),
            ("cargo de", "de la Subsecretaría"),
            ("cargo de", "de la Dirección"),
            ("cargo de", "de la Jefatura"),
            ("cargo de", "del Instituto"),
            ("cargo de", "dependiente"),
            ("cargo de", " de la DIRECCIÓN"),
            ("cargo de", " de la SUBSECRETARÍA"),
            ("cargo de", ", dependiente"),
            ("cargo de", " del MINISTERIO"),
            ("cargo de", " del SISTEMA"),
            ("cargo de", "al"),
            ("cargo de", "a la"),
            ("cargo de", "del"),
            ("cargo de", "en la"),
            ("cargo extraescalafonario de", ", con rango"),
            ("como", ".")
        ]

        tuplas_modificadas = [("para cumplir funciones de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_2 = [("Dáse por designada, a partir del" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_3 = [("Dáse por designado, a partir del" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_4 = [("Desígnase en carácter" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_5 = [("la función de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_6 = [("Desígnase como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_7 = [("2021, como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_8 = [("2020, como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_9 = [("2019, como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_10 = [("2022, como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_11 = [("2023, como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_12 = [("como" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_13 = [("en el cargo" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_14 = [("para cumplir funciones de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_15 = [("de 2019," if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_16 = [("de 2020," if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_17 = [("de 2021," if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_18 = [("de 2022," if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_19 = [("de 2023," if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_20 = [("Desígnase" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_21 = [("de 2019" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_22 = [("de 2020" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_23 = [("de 2021" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_24 = [("de 2022" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_25 = [("de 2023" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_26 = [("asignándosele un puesto de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_27 = [("puesto de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]
        tuplas_modificadas_28 = [("función de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras] #asignándosele el puesto de
        tuplas_modificadas_29 = [("asignándosele el puesto de" if x == "cargo de" else x, y) for x, y in conjuntos_palabras]


        conjuntos_palabras.extend(tuplas_modificadas)
        conjuntos_palabras.extend(tuplas_modificadas_2)
        conjuntos_palabras.extend(tuplas_modificadas_3)
        conjuntos_palabras.extend(tuplas_modificadas_4)
        conjuntos_palabras.extend(tuplas_modificadas_5)
        conjuntos_palabras.extend(tuplas_modificadas_7)
        conjuntos_palabras.extend(tuplas_modificadas_8)
        conjuntos_palabras.extend(tuplas_modificadas_9)
        conjuntos_palabras.extend(tuplas_modificadas_10)
        conjuntos_palabras.extend(tuplas_modificadas_11)
        conjuntos_palabras.extend(tuplas_modificadas_12)
        conjuntos_palabras.extend(tuplas_modificadas_13)
        conjuntos_palabras.extend(tuplas_modificadas_14)
        conjuntos_palabras.extend(tuplas_modificadas_15)
        conjuntos_palabras.extend(tuplas_modificadas_16)
        conjuntos_palabras.extend(tuplas_modificadas_17)
        conjuntos_palabras.extend(tuplas_modificadas_18)
        conjuntos_palabras.extend(tuplas_modificadas_19)
        conjuntos_palabras.extend(tuplas_modificadas_20)
        conjuntos_palabras.extend(tuplas_modificadas_21)
        conjuntos_palabras.extend(tuplas_modificadas_22)
        conjuntos_palabras.extend(tuplas_modificadas_23)
        conjuntos_palabras.extend(tuplas_modificadas_24)
        conjuntos_palabras.extend(tuplas_modificadas_25)
        conjuntos_palabras.extend(tuplas_modificadas_26)
        conjuntos_palabras.extend(tuplas_modificadas_27)
        conjuntos_palabras.extend(tuplas_modificadas_28)
        conjuntos_palabras.extend(tuplas_modificadas_29)

        texto_extraido.append(extraer_texto(parrafo, conjuntos_palabras))

    return texto_extraido

def extraer_texto(parrafo, conjuntos_palabras):
    for conjunto_palabras in conjuntos_palabras:
        inicio = parrafo.find(conjunto_palabras[0])
        fin = parrafo.find(conjunto_palabras[1])

        if inicio != -1 and fin != -1:
            return parrafo[inicio+len(conjunto_palabras[0]):fin].strip()

    return np.nan

df_filtrado_designaciones['Cargo Extraido'] = extraer_texto_entre_palabras(df_filtrado_designaciones['Parrafos_filtrados'])

In [ ]:
#Como la mayor parte de las resoluciones se encuentran sucias (contienen la "secretaría", o el "reporta a", que esta siempre en mayusculas), borramos para cada observación
#las palabras que esten despues de la primera mayuscula
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def borrar_palabras_despues_primer_mayuscula(parrafo):
    palabras = word_tokenize(parrafo)
    for i, palabra in enumerate(palabras):
        if palabra.isupper():
            return ' '.join(palabras[:i]).strip()
    return parrafo
df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].astype(str)


df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].apply(borrar_palabras_despues_primer_mayuscula)

In [ ]:
#Función que hace lo mismo que la anterior, pero con la frase "de la"
def borrar_ultima_frase_de_la(parrafo):
    indice = parrafo.rfind("de la")
    if indice != -1:
        return parrafo[:indice].strip()
    return parrafo


df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].astype(str)

df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].apply(borrar_ultima_frase_de_la)

In [ ]:
#Lo mismo que lo anterior, pero con la frase ", dependiente"
def borrar_ultima_frase_de_la(parrafo):
    indice = parrafo.rfind(", dependiente")
    if indice != -1:
        return parrafo[:indice].strip()
    return parrafo

df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].astype(str)
df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].apply(borrar_ultima_frase_de_la)

In [ ]:
#Lo mismo que lo anterior, pero con la frase "dependiente"
def borrar_ultima_frase_de_la(parrafo):
    indice = parrafo.rfind("dependiente")
    if indice != -1:
        return parrafo[:indice].strip()
    return parrafo

df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].astype(str)
df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].apply(borrar_ultima_frase_de_la)

In [ ]:
#Lo mismo que lo anterior, pero con la frase ", con caracter"
def borrar_ultima_frase_de_la(parrafo):
    indice = parrafo.rfind(", con caracter")
    if indice != -1:
        return parrafo[:indice].strip()
    return parrafo

df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].astype(str)
df_filtrado_designaciones["Cargo Extraido"] = df_filtrado_designaciones["Cargo Extraido"].apply(borrar_ultima_frase_de_la)

In [ ]:
#Reacomodamos manualmente lo mas que podemos especificando los valores que deseamos extraer de la columna "Parrafos_filtrados" para aquellas observaciones sucias. El valor deseado es el que esta despues de los dos puntos
frases_nuevos_valores = {
    "Titular de la UNIDAD GABINETE DE ASESORES": "Titular de la UNIDAD GABINETE DE ASESORES",
    "Titular de la Unidad Gabinete de Asesores": "Titular de la Unidad Gabinete de Asesores",
    "Titular de la UNIDAD EJECUTORA ESPECIAL TEMPORARIA": "Titular de la UNIDAD EJECUTORA ESPECIAL TEMPORARIA",
    "Titular de la Unidad Ejecutora Especial Temporaria": "Titular de la Unidad Ejecutora Especial Temporaria",
    "Abogado Sumariante": "Abogado Sumariante",
    "Referente de Soporte Administrativo": "Referente de Soporte Administrativo",
    "Analista de Comunicación y Contenido Institucional": "Analista de Comunicación y Contenido Institucional",
    "Asesora Legal": "Asesora Legal",
    "Mozo": "Mozo",
    "ARTÍCULO 1°.- Desígnase, a partir del dictado de la presente, a Sabrina Elizabeth SUAREZ (D.N.I. N° 22.518.474), en un cargo de la dotación de planta permanente de la DIRECCIÓN NACIONAL DE CONTROL DE FRONTERAS E HIDROVÍAS dependiente de la SUBSECRETARÍA DE INTERVENCIÓN FEDERAL de la SECRETARÍA DE SEGURIDAD Y POLÍTICA CRIMINAL de este Ministerio, Nivel C, Grado 11, Agrupamiento General, Tramo Avanzado, del Convenio Colectivo de Trabajo Sectorial del personal del Sistema Nacional de Empleo Público (SINEP), homologado por el Decreto N° 2098/08, para cumplir funciones de “Secretaria”": "Secretaria",
    "Adrian Ezequiel ARANDA": "Secretario",
    "Analista de Administración y Gestión del Personal": "Analista de Administración y Gestión del Personal",
    "Director del COLEGIO MAYOR ARGENTINO “NUESTRA SEÑORA DE LUJÁN”": "Director del COLEGIO MAYOR ARGENTINO “NUESTRA SEÑORA DE LUJÁN”",
    "Director del MUSEO MALVINAS E ISLAS DEL ATLÁNTICO SUR": "Director del MUSEO MALVINAS E ISLAS DEL ATLÁNTICO SUR",
    "Director del INSTITUTO DEL SERVICIO EXTERIOR DE LA NACIÓN": "Director del INSTITUTO DEL SERVICIO EXTERIOR DE LA NACIÓN",
    "Director del PALACIO NACIONAL DE LAS ARTES": "Director del PALACIO NACIONAL DE LAS ARTES",
    "Director del MUSEO REGIONAL DE PINTURA “JOSÉ ANTONIO TERRY”": "Director del MUSEO REGIONAL DE PINTURA “JOSÉ ANTONIO TERRY”",
    "Director del INSTITUTO DEL SERVICIO EXTERIOR DE LA NACIÓN": "Director del INSTITUTO DEL SERVICIO EXTERIOR DE LA NACIÓN",
    "Elbio Néstor LAUCIRICA": "Vocal del Directorio",
    "Mercedes del Valle LEAL": "Directora Ejecutiva",
    "Director de Asuntos Políticos del MERCOSUR": "Director de Asuntos Políticos del MERCOSUR",
    "Directora de Asuntos Políticos del MERCOSUR": "Directora de Asuntos Políticos del MERCOSUR",
    "Asesora Profesional": "Asesora Profesional",
    "Directora Nacional de Oferta Académica": "Directora Nacional de Oferta Académica",
    "Jefe del ESTADO MAYOR CONJUNTO DE LAS FUERZAS ARMADAS": "Jefe del ESTADO MAYOR CONJUNTO DE LAS FUERZAS ARMADAS",
    "Jefe del ESTADO MAYOR GENERAL DE LA FUERZA AÉREA ARGENTINA": "Jefe del ESTADO MAYOR GENERAL DE LA FUERZA AÉREA ARGENTINA",
    "Jefe del ESTADO MAYOR GENERAL DE LA ARMADA": "Jefe del ESTADO MAYOR GENERAL DE LA ARMADA",
    "Jefe del ESTADO MAYOR GENERAL DEL EJÉRCITO": "Jefe del ESTADO MAYOR GENERAL DEL EJÉRCITO",
    "Director Nacional de Transformación Digital y Desarrollo Sostenible": "Director Nacional de Transformación Digital y Desarrollo Sostenible",
    "Directora Nacional de Oferta Académica del INSTITUTO NACIONAL DE LA ADMINISTRACIÓN PÚBLICA": "Directora Nacional de Oferta Académica del INSTITUTO NACIONAL DE LA ADMINISTRACIÓN PÚBLICA",
    "Sergio PAZ (MI N° 16.580.800)": "Presidente Alterno de la la COMISIÓN NACIONAL DE TRABAJO Y FISCALIZACIÓN DEL REGIMEN DE DISTRIBUCIÓN Y VENTA DE DIARIOS, REVISTAS Y AFINES",
    "Auditora Adjunta General en PREFECTURA NAVAL ARGENTINA": "Auditora Adjunta General en PREFECTURA NAVAL ARGENTINA",
    "Director Nacional de Asuntos Federales": "Director Nacional de Asuntos Federales",
    "Directora Nacional de Investigación y Desarrollo Profesional Docente": "Directora Nacional de Investigación y Desarrollo Profesional Docente",
    "Director Ejecutivo del INSTITUTO NACIONAL DE EDUCACIÓN TECNOLÓGICA": "Director Ejecutivo del INSTITUTO NACIONAL DE EDUCACIÓN TECNOLÓGICA",
    "Presidente del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS": "Presidente del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS",
    "Directora de Respuesta al VIH, ITS, Hepatitis Virales y Tuberculosis": "Directora de Respuesta al VIH, ITS, Hepatitis Virales y Tuberculosis",
    "Presidenta del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS": "Presidenta del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS",
    "Vicepresidente del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS": "Vicepresidente del INSTITUTO NACIONAL DE ASUNTOS INDÍGENAS",
    "auxiliar administrativa": "auxiliar administrativa",
    "Subsecretario de Gestión Administrativa": "Subsecretario de Gestión Administrativa",
    "Analista Contable": "Analista Contable",
    "Presidenta del Directorio de TELAM SOCIEDAD DEL ESTADO": "Presidenta del Directorio de TELAM SOCIEDAD DEL ESTADO",
    "Ernesto Yoliztly LEJDERMAN AVALOS": "analista",
    "Leonardo Esteban CABRERA DOMINGUEZ": "Subsecretario de Puertos, Vías Navegables y Marina Mercante",
    "Principal-Responsable de Agencia Territorial Viedma": "Principal-Responsable de Agencia Territorial Viedma",
    "chofer en la": "chofer",
    "Director de Asuntos Institucionales del MERCOSUR": "Director de Asuntos Institucionales del MERCOSUR",
    "Responsable de AUDITORÍA DE SISTEMAS DE INNOVACIÓN PÚBLICA": "Responsable de AUDITORÍA DE SISTEMAS DE INNOVACIÓN PÚBLICA",
    "Asistente Administrativa": "Asistente Administrativa",
    "Asistente Administrativo": "Asistente Administrativo",
    "Auxiliar Administrativa": "Auxiliar Administrativa",
    "Directora de Afirmación de los Derechos Indígenas": "Directora de Afirmación de los Derechos Indígenas",
    "en el cargo de Ministra de Salud": "Ministra de Salud",
    "Directora del MUSEO NACIONAL DE ARTE DECORATIVO": "Directora del MUSEO NACIONAL DE ARTE DECORATIVO",
    "Directora del PALACIO SAN JOSÉ - MUSEO Y MONUMENTO NACIONAL “JUSTO JOSÉ DE URQUIZA”": "Directora del PALACIO SAN JOSÉ - MUSEO Y MONUMENTO NACIONAL “JUSTO JOSÉ DE URQUIZA”",
    "Director de Logística": "Director de Logística",
    "Analista Administrativa": "Analista Administrativa",
    "Asesora en la DIRECCIÓN DE CONTABILIDAD Y TESORERÍA": "Asesora en la DIRECCIÓN DE CONTABILIDAD Y TESORERÍA",
    "Director General Técnico Administrativo": "Director General Técnico Administrativo",
    "Jefa de División de Gestión Documental del Departamento de Coordinación Técnica": "Jefa de División de Gestión Documental del Departamento de Coordinación Técnica",
    "Asesora en materia de Derechos Humanos": "Asesora en materia de Derechos Humanos",
    "auxiliar administrativo": "auxiliar administrativo",
    "Directora de Infracciones , Liquidaciones e Intervenciones Judiciales": "Directora de Infracciones , Liquidaciones e Intervenciones Judiciales",
    "Analista Jurídico": "Analista Jurídico",
    "Coordinador de Planificación": "Coordinador de Planificación",
    "Director de Tierras y Registro Nacional de Comunidades Indígenas": "Director de Tierras y Registro Nacional de Comunidades Indígenas",
    "Directora de Desarrollo de Comunidades Indígenas": "Directora de Desarrollo de Comunidades Indígenas",
    "Asistente Administrativo": "Asistente Administrativo",
    "asesora legal": "asesora legal",
    "Secretaria de la Audiencia Pública": "Secretaria de la Audiencia Pública",
    "Coordinadora Nacional del PROGRAMA DE COORDINACIÓN NACIONAL PARA EL COMBATE DEL LAVADO DE ACTIVOS Y LA FINANCIACIÓN DEL TERRORISMO del MINISTERIO DE JUSTICIA Y DERECHOS HUMANO": "Coordinadora Nacional del PROGRAMA DE COORDINACIÓN NACIONAL PARA EL COMBATE DEL LAVADO DE ACTIVOS Y LA FINANCIACIÓN DEL TERRORISMO del MINISTERIO DE JUSTICIA Y DERECHOS HUMANO",
    "Asesora Principal-Responsable de AGENCIA TERRITORIAL ADOLFO GONZÁLES CHAVES": "Asesora Principal-Responsable de AGENCIA TERRITORIAL ADOLFO GONZÁLES CHAVES",
    "Director Técnico Administrativo": "Director Técnico Administrativo",
    "Director Nacional de Cumplimiento y Fiscalización de Cooperativas y Mutuales": "Director Nacional de Cumplimiento y Fiscalización de Cooperativas y Mutuales",
    "Asistente en Planificación y Ejecución de Proyectos en Materia de Derechos Humanos": "Asistente en Planificación y Ejecución de Proyectos en Materia de Derechos Humanos",
    "Coordinador de SUMAR": "Coordinador de SUMAR",
    "Asistente en Gestión Financiera y Contable": "Asistente en Gestión Financiera y Contable",
    "Síndico del INSTITUTO DE AYUDA FINANCIERA PARA PAGO DE RETIROS Y PENSIONES MILITARES": "Síndico del INSTITUTO DE AYUDA FINANCIERA PARA PAGO DE RETIROS Y PENSIONES MILITARES",
    "asesor altamente especializado": "asesor altamente especializado",
    "analista profesional": "analista profesional",
    "profesional especializado en Planificación e Implementación de Políticas Públicas": "profesional especializado en Planificación e Implementación de Políticas Públicas",
    "Directora de Relaciones Institucionales": "Directora de Relaciones Institucionales",
    "Director de Política Comercial con el MERCOSUR y ALADI": "Director de Política Comercial con el MERCOSUR y ALADI",
    "Responsable del PROGRAMA NACIONAL DE REPRODUCCIÓN MÉDICAMENTE ASISTIDA": "Responsable del PROGRAMA NACIONAL DE REPRODUCCIÓN MÉDICAMENTE ASISTIDA",
    "Analista de Auditoría": "Analista de Auditoría",
    "Asesor Principal - Responsable en la AGENCIA TERRITORIAL MORENO": "Asesor Principal - Responsable en la AGENCIA TERRITORIAL MORENO",
    "Moderador de la Audiencia Pública": "Moderador de la Audiencia Pública",
    "Directora Nacional de Cumplimiento y Fiscalización de Cooperativas y Mutuales": "Directora Nacional de Cumplimiento y Fiscalización de Cooperativas y Mutuales",
    "Director Nacional de Desarrollo y Promoción Cooperativo y Mutual": "Director Nacional de Desarrollo y Promoción Cooperativo y Mutual",
    "Analista de Control Interno": "Analista de Control Interno",
    "Director de Afirmación de los Derechos Indígenas": "Director de Afirmación de los Derechos Indígenas",
    "Asistente Administrativo": "Asistente Administrativo",
    "asesora profesional": "asesora profesional",
    "Referente en Gestión de Personal": "Referente en Gestión de Personal",
    "asistente en la DIRECCIÓN DE GESTIÓN DE POLÍTICAS REPARATORIAS": "asistente en la DIRECCIÓN DE GESTIÓN DE POLÍTICAS REPARATORIAS",
    "Coordinador de Actores Territoriales del INSTITUTO NACIONAL DE JUVENTUD": "Coordinador de Actores Territoriales del INSTITUTO NACIONAL DE JUVENTUD",
    "Analista Jurídica": "Analista Jurídica",
    "Asesora Técnica": "Asesora Técnica",
    "Asistente de Comunicación": "Asistente de Comunicación",
    "Analista de Control Interno": "Analista de Control Interno",
    "Nicolás Fernández (D.N.I. N° 36.529.388),": "Secretaría de la Audiencia Pública",
    "asesor especializado": "asesor especializado",
    "Director de Tierras y Registro Nacional de Comunidades Indígenas": "Director de Tierras y Registro Nacional de Comunidades Indígenas",
    "Asistente Administrativo": "Asistente Administrativo",
    "Analista Jurídico": "Analista Jurídico",
    "asesora especializada en políticas de género y diversidad": "asesora especializada en políticas de género y diversidad",
    "Asistente Jurídica": "Asistente Jurídica",
    "abogada en materia de Derechos Humanos": "abogada en materia de Derechos Humanos",
    "abogada especializada en derechos humanos": "abogada especializada en derechos humanos",
    "analista jurídica": "analista jurídica",
    "Asesora Legal Especializada": "Asesora Legal Especializada",
    "asesor legal": "asesor legal",
    "asesora legal especializada": "asesora legal especializada",
    "auxiliar administrativo": "auxiliar administrativo",
    "Asistente Administrativo": "Asistente Administrativo",
    "Auxiliar Administrativa": "Auxiliar Administrativa",
    "en el cargo de Ministro de Salud": "Ministro de Salud"
}

for frase, nuevo_valor in frases_nuevos_valores.items():
    filas_a_cambiar = df_filtrado_designaciones[df_filtrado_designaciones['Parrafos_filtrados'].str.contains(frase)].index
    df_filtrado_designaciones.loc[filas_a_cambiar, 'Cargo Extraido'] = nuevo_valor

In [ ]:
#Repetimos el proceso
frases_nuevos_valores = {
  "Coordinador Región NOA": "Coordinador Región NOA",
  "Coordinador Región CENTRO": "Coordinador Región CENTRO",
  "Coordinadora Región NEA": "Coordinadora Región NEA",
  "Subdirectora Nacional del SERVICIO PENITENCIARIO FEDERAL": "Subdirectora Nacional del SERVICIO PENITENCIARIO FEDERAL",
  "Director Regional NOA": "Director Regional NOA",
  "Director Regional NEA": "Director Regional NEA",
  "Titular de la AUDITORÍA INTERNA ADJUNTA DE PROCESOS DE APOYO": "Titular de la AUDITORÍA INTERNA ADJUNTA DE PROCESOS DE APOYO",
  "Titular de la Unidad de Gabinete de Asesores": "Titular de la Unidad de Gabinete de Asesores",
  "Titular de la OFICINA NACIONAL DE CONTRATACIONES": "Titular de la OFICINA NACIONAL DE CONTRATACIONES",
  "Titular de la AUDITORÍA INTERNA ADJUNTA OPERACIONAL": "Titular de la AUDITORÍA INTERNA ADJUNTA OPERACIONAL",
  "Titular de la UNIDAD DE GESTIÓN DE LA SOSTENIBILIDAD DE LA DEUDA PÚBLICA EXTERNA": "Titular de la UNIDAD DE GESTIÓN DE LA SOSTENIBILIDAD DE LA DEUDA PÚBLICA EXTERNA",
  "Titular de la “UNIDAD EJECUTORA ESPECIAL TEMPORARIA FEDERAL DE FORTALECIMIENTO DEPORTIVO”": "Titular de la “UNIDAD EJECUTORA ESPECIAL TEMPORARIA FEDERAL DE FORTALECIMIENTO DEPORTIVO”",
  "Subsecretario de la Productividad y Desarrollo Regional Pyme": "Subsecretario de la Productividad y Desarrollo Regional Pyme",
  "Subsecretario de la Pequeña y Mediana Empresa": "Subsecretario de la Pequeña y Mediana Empresa",
  "Subadministrador de la UESTEE": "Subadministrador de la Unidad Especial Sistema de Transmisión de Energía Eléctrica (UESTEE)",
  "Subadministrador de la Unidad Especial Sistema de Transmisión de Energía Eléctrica (UESTEE)": "Subadministrador de la Unidad Especial Sistema de Transmisión de Energía Eléctrica (UESTEE)",
  "Jefe de la Agencia Territorial CIUDAD AUTÓNOMA DE BUENOS AIRES": "Jefe de la Agencia Territorial CIUDAD AUTÓNOMA DE BUENOS AIRES",
  "Director de la FUNDACIÓN ARGENTINA EN LA CIUDAD INTERNACIONAL UNIVERSITARIA DE PARÍS, REPÚBLICA FRANCESA": "Director de la FUNDACIÓN ARGENTINA EN LA CIUDAD INTERNACIONAL UNIVERSITARIA DE PARÍS, REPÚBLICA FRANCESA",
  "Directora Nacional de la Oficina Nacional de Innovación de Gestión": "Directora Nacional de la Oficina Nacional de Innovación de Gestión",
  "Secretario de la Pequeña y Mediana Empresa y los Emprendedores": "Secretario de la Pequeña y Mediana Empresa y los Emprendedores",
  "Titular del INSTITUTO NACIONAL DE CAPACITACIÓN POLÍTICA": "Titular del INSTITUTO NACIONAL DE CAPACITACIÓN POLÍTICA",
  "Vocal del Comité de Administración del Fondo Fiduciario para el Transporte Eléctrico Federal (FFTEF)": "Vocal del Comité de Administración del Fondo Fiduciario para el Transporte Eléctrico Federal (FFTEF)",
  "Secretario Técnico del COMITÉ CONSULTIVO PERMANENTE DE LA LEY DE RIESGOS DEL TRABAJO": "Secretario Técnico del COMITÉ CONSULTIVO PERMANENTE DE LA LEY DE RIESGOS DEL TRABAJO",
  "subtitular de la Unidad Ejecutora Especial Temporaria “Unidad de Articulación Estratégica de Áreas Económicas y Productivas”": "subtitular de la Unidad Ejecutora Especial Temporaria “Unidad de Articulación Estratégica de Áreas Económicas y Productivas”",
  "Coordinador Técnico de la DIRECCIÓN GENERAL DE COORDINACIÓN INSTITUCIONAL": "Coordinador Técnico de la DIRECCIÓN GENERAL DE COORDINACIÓN INSTITUCIONAL",
  "Daniela Celeste AMBROSI (D.N.I. N° 31.369.844) para cumplir funciones de Analista Técnica": "Analista Técnica",
  "ANALISTA DE APLICACIÓN CIENTÍFICA CON ORIENTACIÓN EN AVISOS MARÍTIMOS": "ANALISTA DE APLICACIÓN CIENTÍFICA CON ORIENTACIÓN EN AVISOS MARÍTIMOS",
  "Asistente Administrativo": "Asistente Administrativo",
  "Coordinador de las Leyes N° 24.043, N° 26.564 y Exilio Forzado": "Coordinador de las Leyes N° 24.043, N° 26.564 y Exilio Forzado",
  "Coordinador REPRO": "Coordinador REPRO",
  "Director Nacional de la OFICINA NACIONAL DE TECNOLOGÍAS DE INFORMACIÓN": "Director Nacional de la OFICINA NACIONAL DE TECNOLOGÍAS DE INFORMACIÓN",
  "Directora Ejecutiva del INSTITUTO NACIONAL DE FORMACIÓN DOCENTE": "Directora Ejecutiva del INSTITUTO NACIONAL DE FORMACIÓN DOCENTE",
  "Coordinador Regional NEA": "Coordinador Regional NEA",
  "Coordinador Regional NOA-NEA": "Coordinador Regional NOA-NEA",
  "Coordinador Regional NOA": "Coordinador Regional NOA",
  "Desígnase en carácter de Secretaría de la Audiencia Pública": "Secretaría de la Audiencia Pública",
  "Subtitular de la Unidad Ejecutora Especial Temporaria": "Subtitular de la Unidad Ejecutora Especial Temporaria",
  "Presidenta de la COMISIÓN CASCOS BLANCOS": "Presidenta de la COMISIÓN CASCOS BLANCOS",
  "Director Nacional del SERVICIO PENITENCIARIO FEDERAL": "Director Nacional del SERVICIO PENITENCIARIO FEDERAL",
  "Coordinadora de la Comisión del Trabajo Rural – Agrario": "Coordinadora de la Comisión del Trabajo Rural – Agrario",
  "Coordinadora de las Leyes N° 24.321, N° 24.411 y N° 25.914": "Coordinadora de las Leyes N° 24.321, N° 24.411 y N° 25.914",
  "Administrador de la UNIDAD ESPECIAL SISTEMA DE TRANSMISIÓN DE ENERGÍA ELÉCTRICA (UESTEE)": "Administrador de la UNIDAD ESPECIAL SISTEMA DE TRANSMISIÓN DE ENERGÍA ELÉCTRICA (UESTEE)",
  "Coordinadora Ley N° 26.913": "Coordinadora Ley N° 26.913",
  "Auditor de la AUDITORÍA ADJUNTA EN POLICÍA FEDERAL ARGENTINA": "Auditor de la AUDITORÍA ADJUNTA EN POLICÍA FEDERAL ARGENTINA",
  "Auditor de la AUDITORIA ADJUNTA GENERAL EN POLICÍA FEDERAL ARGENTINA": "Auditor de la AUDITORIA ADJUNTA GENERAL EN POLICÍA FEDERAL ARGENTINA",
  "Coordinadora Regional NOA-NEA": "Coordinadora Regional NOA-NEA",
  "Coordinadora Regional NOA - NEA": "Coordinadora Regional NOA - NEA",
  "Asesora Especializada de la UNIDAD GABINETE DE ASESORES": "Asesora Especializada de la UNIDAD GABINETE DE ASESORES",
  "Directora Operativa de la UNIDAD GABINETE DE ASESORES": "Directora Operativa de la UNIDAD GABINETE DE ASESORES",
  "en el cargo de Ministra de las Mujeres, Géneros y Diversidad": "Ministra de las Mujeres, Géneros y Diversidad",
  "titular de la UNIDAD DE COMUNICACIÓN DE GESTIÓN DEL/DE LA JEFE/A DE GABINETE DE MINISTROS": "titular de la UNIDAD DE COMUNICACIÓN DE GESTIÓN DEL/DE LA JEFE/A DE GABINETE DE MINISTROS",
  "titular de la UNIDAD GABINETE DE ASESORES": "titular de la UNIDAD GABINETE DE ASESORES",
  "titular de la Unidad Ejecutora Especial Temporaria": "titular de la Unidad Ejecutora Especial Temporaria",
  "titular de la Delegación Técnico-Administrativa de la SECRETARÍA DE DEPORTES": "titular de la Delegación Técnico-Administrativa de la SECRETARÍA DE DEPORTES",
  "titular de la Unidad Ejecutora Especial Temporaria “Unidad de Articulación Estratégica de Áreas Económicas y Productivas”": "titular de la Unidad Ejecutora Especial Temporaria “Unidad de Articulación Estratégica de Áreas Económicas y Productivas”",
  "Asesora Técnica en la Coordinación de Vinculación con Unidades Científicas y Académicas y Articulación Jurisdiccional": "Asesora Técnica en la Coordinación de Vinculación con Unidades Científicas y Académicas y Articulación Jurisdiccional",
  "Asesora Técnica en el MINISTERIO DE SEGURIDAD": "Asesora Técnica en el MINISTERIO DE SEGURIDAD",
  "Comandante Operacional del ESTADO MAYOR CONJUNTO DE LAS FUERZAS ARMADAS": "Comandante Operacional del ESTADO MAYOR CONJUNTO DE LAS FUERZAS ARMADAS",
  "Desígnase a la Licenciada María Candelaria QUIROGA": "Vocal del Comité de Administración del Fondo Fiduciario para el Transporte Eléctrico Federal (FFTEF)",
  "a la licenciada Daniela Celeste AMBROSI": "Analista Técnica",
  "al Ingeniero Electromecánico Miguel Ángel MAZZA CAMPOS": "Administrador de la UNIDAD ESPECIAL SISTEMA DE TRANSMISIÓN DE ENERGÍA ELÉCTRICA (UESTEE)",
  "Investigador con orientación en biología marina y ambiente marino": "Investigador con orientación en biología marina y ambiente marino",
  "Directora Vocal del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)": "Directora Vocal del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)",
  "Técnico de Control / Inspector / Verificador con orientación en Fiscalización en Comercialización de Carnes, Granos y Lácteos": "Técnico de Control / Inspector / Verificador con orientación en Fiscalización en Comercialización de Carnes, Granos y Lácteos",
  "Analista de Planificación e Implementación de Políticas Públicas con orientación en Bioeconomía y Desarrollo Regional": "Analista de Planificación e Implementación de Políticas Públicas con orientación en Bioeconomía y Desarrollo Regional",
  "Presidente del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)": "Presidente del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)",
  "Vicepresidente Ejecutivo del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)": "Vicepresidente Ejecutivo del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)",
  "Analista de Presupuesto y Finanzas": "Analista de Presupuesto y Finanzas",
  "Analista de Planificación e Implementación de Políticas Públicas con orientación en Sector Agroalimentario": "Analista de Planificación e Implementación de Políticas Públicas con orientación en Sector Agroalimentario",
  "Subsecretario de Política Económica y Financiera de Transporte": "Subsecretario de Política Económica y Financiera de Transporte",
  "Director Vocal del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)": "Director Vocal del Directorio de la Empresa del Estado “CONSTRUCCION DE VIVIENDA PARA LA ARMADA” (COVIARA)",
  "Secretaria de Articulación de Política Social": "Secretaria de Articulación de Política Social",
  "Secretaria de Articulación Interjurisdiccional": "Secretaria de Articulación Interjurisdiccional",
  "Subsecretaria de Políticas de Diversidad": "Subsecretaria de Políticas de Diversidad"

}

for frase, nuevo_valor in frases_nuevos_valores.items():
    filas_a_cambiar = df_filtrado_designaciones[df_filtrado_designaciones['Parrafos_filtrados'].str.contains(frase)].index
    df_filtrado_designaciones.loc[filas_a_cambiar, 'Cargo Extraido'] = nuevo_valor

#Profesión

In [86]:
#Buscamos palabras clave referidas a profesión y las metemos en la columna "Profesión" // no es sensible a mayusculas o minusculas
import nltk
from nltk.tokenize import word_tokenize
palabras_clave = [
    'señor', 'señora',
    'licenciado', 'licenciada',
    'contador público', 'contadora pública',
    'ingeniero', 'ingeniera',
    'abogado', 'abogada',
    'doctor', 'doctora',
    'analista', 'analista',
    'embajador plenipotenciario', 'embajadora plenipotenciaria',
    'arquitecto', 'arquitecta',
    'maestro', 'maestra',
    'profesor', 'profesora',
    'ingeniero', 'ingeniera',
    'científico', 'científica',
    'programador', 'programadora',
    'piloto', 'pilota',
    'escritor', 'escritora',
    'artista', 'artista',
    'diseñador', 'diseñadora',
    'psicólogo', 'psicóloga',
    'enfermero', 'enfermera',
    'dentista', 'dentista',
    "agente", "agrónomo", "agrónoma"
    "contador", "contadora", "arq.", "dr.", "dra.",
    "Médico", "Médica", "informático", "informática", "técnica", "técnico", "magíster", "magistrada", "magistrado",
    "Agrimensor", "Agrimensora", "Mg.", "psicopedagoga", "psicopedagogo", "procurador", "procuradora", "General de Brigada",
    "Brigadier", "Contraalmirante", "CPN", "General de División", "sr.", "sra.", "bioquímico", "bioquímica"
]
def buscar_palabras_clave(parrafo, palabras_clave):
    if isinstance(parrafo, str):
        resultados = []
        for palabra in palabras_clave:
            if re.search(r'\b{}\b'.format(palabra), parrafo, re.IGNORECASE):
                resultados.append(palabra)
        if resultados:
            return resultados
        else:
            return [np.nan]
    else:
        return [np.nan]
df_filtrado_designaciones['Profesión'] = df_filtrado_designaciones['Parrafos_filtrados'].apply(lambda x: buscar_palabras_clave(x, palabras_clave))

#Genero

In [87]:
#Importamos csv del gobierno que tiene los generos asignados a los DNI
data=pd.read_csv("BIME_estructura_autoridades_apn_20231209 (1).csv")
df_bime=pd.DataFrame(data)

In [88]:
df_bime['autoridad_dni'] = df_bime['autoridad_dni'].fillna(-1).astype(int).astype(str).replace('-1', 'NaN') #Eliminamos el ".0" al final de cada DNI

In [89]:
#Mapeamos los valores de genero asignado a cada DNI a nuestro DF
diccionario_genero = dict(zip(df_bime['autoridad_dni'], df_bime['autoridad_sexo']))
diccionario_genero = {str(key): value for key, value in diccionario_genero.items()}
df_filtrado_designaciones['Genero'] = df_filtrado_designaciones['DNI Extraido'].astype(str).map(diccionario_genero)

In [90]:
!pip install gender-guesser #Utilizamos Gender Guesser para completar los DNI que son exclusivos a nuestro df y que por ende no tienen genero asignado

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 3.0 MB/s eta 0:00:00


In [91]:
lista_nombres_genero=df_filtrado_designaciones["Nombre Completo"].tolist()
primeras_palabras = [nombre.split()[0] if isinstance(nombre, str) else np.nan for nombre in lista_nombres_genero] #Creamos una lista que contenga solo los nombres de los funcionarios

In [92]:
import gender_guesser.detector as gender #Aplicamos Gender Guesser a esta lista de nombres

def obtener_generos(nombres):
    detector = gender.Detector()
    generos = []

    for nombre in nombres:
        if isinstance(nombre, str):
            genero = detector.get_gender(nombre)
            generos.append(genero)
        else:
            generos.append(np.nan)

    return generos
resultados = obtener_generos(primeras_palabras)

In [93]:
data={"Nombre Completo": lista_nombres_genero, "Nombre": primeras_palabras, "Genero": resultados}
df_genero=pd.DataFrame(data)
df_genero['Genero'] = df_genero['Genero'].replace({'male': 'M', 'mostly_male': 'M', 'female': 'F', 'mostly_female': 'F'}) #Reemplazamos con los valores deseados (solo M y F)

In [94]:
#Agregamos los resultados
df_filtrado_designaciones['Genero'] = df_filtrado_designaciones.apply(lambda row: df_genero[df_genero['Nombre Completo'] == row['Nombre Completo']]['Genero'].values[0] if pd.isna(row['Genero']) and not df_genero[df_genero['Nombre Completo'] == row['Nombre Completo']].empty else row['Genero'], axis=1)

In [95]:
#Reasignamos manualmente algunos valores que se equivoca el programa
nombres_generos_unknowns = {
    'Demian Mariano Rigamonti': 'M',
    'Simòn Alberto Dasenchich': 'M',
    'Hernan Flavio Poymulle': 'M',
    'Analia Rosa Rodriguez': 'F',
    'Guillermina Boga': 'F',
    'Joao Sebastiao Nieto': 'M',
    'Hernan Pablo Dearriba': 'M',
    'Grisela Alejandra Garcia Ortiz': 'F',
    'Dardo Hernán Perez': 'M',
    'Hernan Carlos Goncebat': 'M',
    'Rocio Ines Rodriguez Lopez': 'F',
    'Analia Judith Baum': 'F',
    'Analia Marisa Garcia': 'F',
    'Indiana Maria Lopez Alonso': 'F',
    'Haroldo Arián Montagu': 'M',
    'Jesica Priscila Franchi': 'F',
    'Analia Gonzalez': 'F',
    'Hernan Rachid': 'M',
    'Nadab Rajzman': 'M',
    'Nair Belen Castillo': 'F',
    'Nahuel Sosa': 'M',
    'Yamila Ines Sirimarco': 'F',
    'Rocio Roche': 'F',
    'Belen Maria Mallaviabarrena': 'F',
    'Yamila Belen Tobler': 'F',
    'Jesica Daniela Ocampo': 'F',
    'Mariangel Ghilardi Sierra': 'F',
    'Favio Ariel Appezzatto': 'M',
    'Belkyss Contino': 'F',
    'Ailen Ratti': 'F',
    'Aldana Armendariz': 'F',
    'Azul Milicevic Gotti': 'F',
    'Danhel Bruna': 'M',
    'Rocio Amelia Pages': 'F',
    'Braian Francisco Laportilla': 'M',
    'Abril Chantal Collada': 'F'
}

df_filtrado_designaciones = df_filtrado_designaciones.reset_index(drop=True)

generos_asignados = df_filtrado_designaciones['Nombre Completo'].map(nombres_generos_unknowns)

df_filtrado_designaciones.loc[generos_asignados.notna(), 'Genero'] = generos_asignados

In [96]:
#Y para cerrar tema genero, si un nombre termina con "a", este siempre debera ser "F"
def cambiar_valor(row):
    nombre_completo = str(row['Nombre Completo'])
    palabras = nombre_completo.split()
    ultima_letra = palabras[0][-1]
    if ultima_letra == "a":
        return "F"
    else:
        return row['Genero']


df_filtrado_designaciones['Genero'] = df_filtrado_designaciones.apply(cambiar_valor, axis=1)

In [97]:
df_filtrado_designaciones.head()

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Edad,Provincia,Ciudad,Via Excepción,Designación Transitoria,Planta Permanente,Venc_Desig_prorr,Grado y Nivel,Profesión,Genero
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,72.0,Formosa,Formosa,True,True,False,2024-03-30,Nivel B - Grado 0,"[contadora pública, contadora]",F
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,71.0,Santa Fe,No Encontrado,True,True,False,2021-04-12,Nivel B - Grado 0,"[ingeniero, ingeniero]",M
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,72.0,Ciudad De Buenos Aires,Palermo,False,False,True,NaT,Nivel A - Grado 8,"[científico, agente, técnico]",M
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,72.0,Ciudad De Buenos Aires,Caballito,True,True,False,2021-10-16,Nivel B - Grado 0,[licenciada],F
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,72.0,Ciudad De Buenos Aires,Palermo,True,True,True,2021-02-09,Nivel B - Grado 11,[licenciada],F


#Agregando reporta a y secretaria

###Secretaría

In [98]:
!pip install PyMuPDF
import fitz
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 23.4 MB/s eta 0:00:00


In [99]:
pdf_document = "SECRETARIAS.pdf" #PDF con las secretarias que cree en base a nuestros organigramas
doc = fitz.open(pdf_document)

text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text += page.get_text()

In [100]:
texto = text

# Dividir el texto en líneas
lineas = texto.strip().split('\n')

# Función para verificar si una línea está completamente en mayúsculas
def esta_en_mayusculas(linea):
    return linea.isupper()

# Filtrar las líneas que están completamente en mayúsculas
lineas_filtradas = [linea for linea in lineas if esta_en_mayusculas(linea)]

# Unir las líneas filtradas nuevamente en un solo texto
texto_filtrado = '\n'.join(lineas_filtradas)

In [101]:
palabras_clave = lineas_filtradas
def encontrar_hits(texto, palabras_clave):
    hits = []
    for palabra in palabras_clave:
        if palabra.lower() in texto.lower():
            hits.append(palabra)
    return hits

# Aplicar la función a cada fila del DataFrame y crear una nueva columna 'hits'
df_filtrado_designaciones['hits'] = df_filtrado_designaciones['Parrafos_filtrados'].apply(lambda x: encontrar_hits(x, palabras_clave))

In [102]:
#Nos quedamos con el ultimo match en ocasión de encontrar dos secretarías, ya que esta tendera a ser la secretaría "superior". La metemos en la columna "ultimo_elemento"
def obtener_ultimo_elemento(lista):
    if len(lista) > 0:
        return lista[-1]
    else:
        return None

df_filtrado_designaciones['ultimo_elemento'] = df_filtrado_designaciones['hits'].apply(obtener_ultimo_elemento)

In [103]:
#Hasta ahora tenemos las secretarias que tenemos en cuenta en nuestro organigrama, pero nos faltan:
  #1) Reporta a
  #2) Secretarias que no estan en los organigramas
#Como sabemos que estas frases estan en mayusculas, creamos una función que extraiga todos los conjuntos de palabras que esten en mayusculas

def extract_uppercase_phrases(text):
    phrases = re.findall(r'\b[A-ZÁÉÍÓÚÑÜ][A-ZÁÉÍÓÚÑÜ,\s]*[A-ZÁÉÍÓÚÑÜ]\b', text)
    return phrases

df_filtrado_designaciones['uppercase_phrases'] = df_filtrado_designaciones['Parrafos_filtrados'].apply(extract_uppercase_phrases)

In [104]:
lista=df_filtrado_designaciones["uppercase_phrases"].tolist()
palabras_clave = ['DIRECCIÓN', 'DIRECCION', 'SUBSECRETARÍA', "INSTITUTO", "SECRETARÍA", "UNIDAD", "SECRETARIA", "SUBSECRETARIA"]#Como esta lista contiene todos los elementos de cada
                                                                                                                                #resolución en mayusculas (como DIAS), es necesario limpiarlos.
                                                                                                                                #Especificamos que tienen que arrancar con una de las siguientes palabras

def filtrar_por_palabra_clave(sublista, palabras_clave):
    return [item for item in sublista if any(item.startswith(p) for p in palabras_clave)]

df_filtrado_designaciones['filtrado'] = df_filtrado_designaciones['uppercase_phrases'].apply(filtrar_por_palabra_clave, palabras_clave=palabras_clave)

def obtener_ultimo_elemento(lista): #Extraemos el ultimo elemento de esta lista de palabras mayusculas, la cual sera la secretaría, subsecretaría o dirección a la que reporte el funcionario
    if len(lista) > 0:
        return lista[-1]
    else:
        return None

df_filtrado_designaciones['ultimatum'] = df_filtrado_designaciones['filtrado'].apply(obtener_ultimo_elemento)

In [105]:
lista=df_filtrado_designaciones["uppercase_phrases"].tolist()
palabras_clave = ['DIRECCIÓN', 'DIRECCION', 'SUBSECRETARÍA', "INSTITUTO", "SECRETARÍA", "UNIDAD", "SECRETARIA", "SUBSECRETARIA", "MINISTERIO", "JEFATURA"]#Repetimos el proceso, pero agregando "ministerio" y "jefatura", para aquellas observaciones
                                                                                                                                                          #en las que no encontramos "secretaría", "subsecretaría", etc

# Función para filtrar elementos que comienzan con alguna palabra clave de la lista
def filtrar_por_palabra_clave(sublista, palabras_clave):
    return [item for item in sublista if any(item.startswith(p) for p in palabras_clave)]

# Aplicar la función a la columna 'lista_de_listas' del DataFrame
df_filtrado_designaciones['filtrado'] = df_filtrado_designaciones['uppercase_phrases'].apply(filtrar_por_palabra_clave, palabras_clave=palabras_clave)

def obtener_ultimo_elemento(lista):
    if len(lista) > 0:
        return lista[-1]
    else:
        return None

df_filtrado_designaciones['ultimatum_2'] = df_filtrado_designaciones['filtrado'].apply(obtener_ultimo_elemento)

In [106]:
#Rellenamos los nans de la lista de matches con los elementos de los organigramas, los elementos que contienen "ministerio" y "jefatura", y por ultimo, si no hay ningun match, con el nombre del ministerio
df_filtrado_designaciones['ultimatum'] = df_filtrado_designaciones['ultimatum'].fillna(df_filtrado_designaciones['ultimo_elemento'])
df_filtrado_designaciones['ultimatum'] = df_filtrado_designaciones['ultimatum'].fillna(df_filtrado_designaciones['ultimatum_2'])
df_filtrado_designaciones['ultimatum'] = df_filtrado_designaciones['ultimatum'].fillna(df_filtrado_designaciones['first_min'])

###Reporta a

Repetimos el mismo proceso, pero extrayendo la primer palabra en mayusculas

In [107]:
def extract_uppercase_phrases(text):
    phrases = re.findall(r'\b[A-ZÁÉÍÓÚÑÜ][A-ZÁÉÍÓÚÑÜ,\s]*[A-ZÁÉÍÓÚÑÜ]\b', text)
    return phrases
df_filtrado_designaciones['uppercase_phrases'] = df_filtrado_designaciones['Parrafos_filtrados'].apply(extract_uppercase_phrases)

In [108]:
lista=df_filtrado_designaciones["uppercase_phrases"].tolist()

In [109]:
palabras_clave = ['DIRECCIÓN', 'DIRECCION', 'SUBSECRETARÍA', "INSTITUTO", "SECRETARÍA", "UNIDAD", "JEFATURA", "MINISTERIO", "SECRETARIA", "SUBSECRETARIA"]

def filtrar_por_palabra_clave(sublista, palabras_clave):
    return [item for item in sublista if any(item.startswith(p) for p in palabras_clave)]

df_filtrado_designaciones['filtrado'] = df_filtrado_designaciones['uppercase_phrases'].apply(filtrar_por_palabra_clave, palabras_clave=palabras_clave)

In [110]:
def obtener_primer_elemento(lista):
    if len(lista) > 0:
        return lista[0]
    else:
        return None

df_filtrado_designaciones['primer_elemento'] = df_filtrado_designaciones['filtrado'].apply(obtener_primer_elemento)

In [111]:
#Agregamos las columnas "reporta a" y "secretaría"
reporta_a=df_filtrado_designaciones['primer_elemento'].tolist()
df_filtrado_designaciones['Reporta a'] = reporta_a

secretaria=df_filtrado_designaciones['ultimatum'].tolist()
df_filtrado_designaciones['Secretaría'] = secretaria
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Genero,hits,ultimo_elemento,uppercase_phrases,filtrado,ultimatum,ultimatum_2,primer_elemento,Reporta a,Secretaría
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,F,"[SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA, SECR...",SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"[ARTÍCULO, CIENTO OCHENTA, MEZA, DIRECCIÓN GEN...","[DIRECCIÓN GENERAL DE ADMINISTRACIÓN, SUBSECRE...",SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,MINISTERIO DE SALUD,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,M,[],None,"[ARTÍCULO, CIENTO OCHENTA, TERÁN, DIRECCIÓN NA...","[DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGEN...",SECRETARÍA DE INNOVACIÓN PÚBLICA,JEFATURA DE GABINETE DE MINISTROS,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...","DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,M,[SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO],SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,"[ARTÍCULO, SECRETARÍA DE GESTIÓN Y EMPLEO PÚBL...","[SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO, JEFAT...",SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,MINISTERIO DE ECONOMÍA,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,F,[SECRETARÍA DE COORDINACIÓN],SECRETARÍA DE COORDINACIÓN,"[ARTÍCULO, CIENTO OCHENTA, GARCÍA, DIRECCIÓN G...","[DIRECCIÓN GENERAL DE RECURSOS HUMANOS, SUBSEC...",SECRETARÍA DE COORDINACIÓN,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,F,[SECRETARÍA DE COORDINACIÓN],SECRETARÍA DE COORDINACIÓN,"[ARTÍCULO, DIRECCIÓN NACIONAL DE COOPERACIÓN Y...",[DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAM...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,MINISTERIO DE AMBIENTE Y DESARROLLO SUSTENTABLE,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,F,[],None,"[ARTÍCULO, CIENTO OCHENTA, OSANO CERDEÑA, DIRE...",[DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE ...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,MINISTERIO DE TRANSPORTE,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,M,[],None,"[ARTÍCULO, CIENTO OCHENTA, YAHARI VÁZQUEZ, MIN...",[MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS],MINISTERI

In [112]:
columnas_basura = ['hits', 'ultimo_elemento', 'uppercase_phrases', 'filtrado', 'ultimatum', 'ultimatum_2', 'primer_elemento']
df_filtrado_designaciones = df_filtrado_designaciones.drop(columns=columnas_basura)

In [113]:
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Ciudad,Via Excepción,Designación Transitoria,Planta Permanente,Venc_Desig_prorr,Grado y Nivel,Profesión,Genero,Reporta a,Secretaría
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,Formosa,True,True,False,2024-03-30,Nivel B - Grado 0,"[contadora pública, contadora]",F,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,No Encontrado,True,True,False,2021-04-12,Nivel B - Grado 0,"[ingeniero, ingeniero]",M,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,Palermo,False,False,True,NaT,Nivel A - Grado 8,"[científico, agente, técnico]",M,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,Caballito,True,True,False,2021-10-16,Nivel B - Grado 0,[licenciada],F,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,Palermo,True,True,True,2021-02-09,Nivel B - Grado 11,[licenciada],F,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,Villa Crespo,True,True,False,2023-12-04,Nivel B - Grado 0,[doctora],F,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,Paso Del Rey,True,True,False,2023-12-23,Nivel C - Grado 0,"[señor, informático]",M,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,No Encontrado,True,True,False,2022-09-14,Nivel C - Grado 0,"[señora, analista, analista]",F,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL..."
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,No Encontrado,True,True,False,2021-11-06,Nivel A - Grado 0,"[licenciada, técnica]",F,UNIDAD GABINETE DE ASESORES,UNIDAD GABINETE DE ASESORES


#Cargos Volumen II

###Cargos con comillas

In [114]:
def extract_quoted_words(text):
    return re.findall(r'"(.*?)"|“(.*?)”', text)

df_filtrado_designaciones['quoted_words'] = df_filtrado_designaciones['Parrafos_filtrados'].apply(lambda x: [match for match in sum(extract_quoted_words(x), ()) if match])
df_non_empty = df_filtrado_designaciones[df_filtrado_designaciones['quoted_words'].apply(lambda x: len(x) > 0)]

In [115]:
df_non_empty['quoted_words'] = df_non_empty['quoted_words'].apply(lambda x: x[0] if len(x) > 0 else None)

<ipython-input-115-a0754de6c2c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['quoted_words'] = df_non_empty['quoted_words'].apply(lambda x: x[0] if len(x) > 0 else None)


In [116]:
phrase_key = 'titular'
df_non_empty['contains_titular'] = df_non_empty['Parrafos_filtrados'].apply(
    lambda x: 1 if phrase_key.lower() in [word.lower() for word in x.split()] else 0)

phrase_key = 'subtitular'
df_non_empty['contains_subtitular'] = df_non_empty['Parrafos_filtrados'].apply(
    lambda x: 1 if phrase_key.lower() in [word.lower() for word in x.split()] else 0)

<ipython-input-116-8bd5678c853b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['contains_titular'] = df_non_empty['Parrafos_filtrados'].apply(
<ipython-input-116-8bd5678c853b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['contains_subtitular'] = df_non_empty['Parrafos_filtrados'].apply(


In [117]:
phrase_to_insert = 'Titular de la Unidad Ejecutora Especial Temporaria '

# Aplicar la función solo a las filas donde 'A' sea igual a 1
df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'quoted_words'] = df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'quoted_words'].apply(lambda x: f'"{x}"') #Agregamos comillas a los nombres de las unidades ejecutoras especiales
df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'combined_titular'] = phrase_to_insert + df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'quoted_words']

phrase_to_insert = 'Subtitular de la Unidad Ejecutora Especial Temporaria '

# Aplicar la función solo a las filas donde 'A' sea igual a 1
df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'quoted_words'] = df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'quoted_words'].apply(lambda x: f'"{x}"')
df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'combined_subtitular'] = phrase_to_insert + df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'quoted_words']

<ipython-input-117-cc0940a551ad>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'combined_titular'] = phrase_to_insert + df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'quoted_words']
<ipython-input-117-cc0940a551ad>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'combined_subtitular'] = phrase_to_insert + df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'quoted_words']


In [118]:
df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'quoted_words'] = df_non_empty.loc[df_non_empty['contains_titular'] == 1, 'combined_titular']
df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'quoted_words'] = df_non_empty.loc[df_non_empty['contains_subtitular'] == 1, 'combined_subtitular']

###Cargos sin comillas

In [119]:
df_empty = df_filtrado_designaciones[df_filtrado_designaciones['quoted_words'].apply(lambda x: len(x) == 0)]

In [120]:
df_empty_filtered = df_empty.dropna(subset=['Reporta a'])

In [121]:
def extract_between_keywords(text, start_keyword, end_keyword):
    # Construir la expresión regular para buscar entre las palabras clave
    pattern = re.compile(rf'{re.escape(start_keyword)}(.*?){re.escape(end_keyword)}')
    # Buscar todas las coincidencias
    matches = pattern.findall(text)
    return matches

# Aplicar la función a cada fila del DataFrame
df_empty_filtered['extracted_section'] = df_empty_filtered.apply(lambda row: extract_between_keywords(row['Parrafos_filtrados'], 'cargo', row['Reporta a']), axis=1)

# Limpiar la lista de resultados
df_empty_filtered['extracted_section'] = df_empty_filtered['extracted_section'].apply(lambda x: x[0] if x else '')

<ipython-input-121-58e34252a623>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['extracted_section'] = df_empty_filtered.apply(lambda row: extract_between_keywords(row['Parrafos_filtrados'], 'cargo', row['Reporta a']), axis=1)
<ipython-input-121-58e34252a623>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['extracted_section'] = df_empty_filtered['extracted_section'].apply(lambda x: x[0] if x else '')


In [122]:
def remove_before_first_capital(text):
    words = text.split()  # Dividir el texto en palabras
    for i, word in enumerate(words):
        if word[0].isupper():  # Verificar si la primera letra de la palabra es mayúscula
            return ' '.join(words[i:])  # Devolver la frase desde la primera palabra con mayúscula
    return text  # Si no se encuentra ninguna palabra con mayúscula, devolver el texto original


df_empty_filtered['processed_text'] = df_empty_filtered['extracted_section'].apply(remove_before_first_capital)

def remove_after_last_capital(text):
    words = text.split()  # Dividir el texto en palabras
    for i in range(len(words)-1, -1, -1):  # Recorrer las palabras de derecha a izquierda
        if words[i][0].isupper():  # Verificar si la primera letra de la palabra es mayúscula
            return ' '.join(words[:i+1])  # Devolver la frase hasta la última palabra con mayúscula
    return text  # Si no se encuentra ninguna palabra con mayúscula, devolver el texto original



# Aplicar la función a la columna del DataFrame
df_empty_filtered['processed_text'] = df_empty_filtered['processed_text'].apply(remove_after_last_capital)

<ipython-input-122-25779e2ae8c0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['processed_text'] = df_empty_filtered['extracted_section'].apply(remove_before_first_capital)
<ipython-input-122-25779e2ae8c0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['processed_text'] = df_empty_filtered['processed_text'].apply(remove_after_last_capital)


In [123]:
phrase_key = 'Titular'
df_empty_filtered['contains_Titular'] = df_empty_filtered['processed_text'].apply(lambda x: 1 if phrase_key in x else 0)

<ipython-input-123-d195fc7862b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['contains_Titular'] = df_empty_filtered['processed_text'].apply(lambda x: 1 if phrase_key in x else 0)


In [124]:
phrase_to_insert = ' de la '

df_empty_filtered['combined'] = np.nan

df_empty_filtered.loc[df_empty_filtered['contains_Titular'] == 1, 'combined'] = df_empty_filtered.loc[df_empty_filtered['contains_Titular'] == 1, 'processed_text'] + phrase_to_insert + df_empty_filtered.loc[df_empty_filtered['contains_Titular'] == 1, 'Reporta a']
df_empty_filtered['combined'] = df_empty_filtered['combined'].fillna(np.nan)

<ipython-input-124-d6cac313a61e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['combined'] = np.nan
<ipython-input-124-d6cac313a61e>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la Auditoría Interna Adjunta de Innovación Pública de la UNIDAD DE AUDITORÍA INTERNA'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular de la UNIDAD GABINETE DE ASESORES'
 'Titular del Instituto Nacional de la Administración Pública de la

##Unificando todo

###Añadiendo columna cargo a cada df

In [125]:
df_non_empty['Cargo'] = np.nan
df_non_empty['Cargo']=df_non_empty['Cargo'].fillna(df_non_empty['combined_titular'])
df_non_empty['Cargo']=df_non_empty['Cargo'].fillna(df_non_empty['combined_subtitular'])
df_non_empty['Cargo']=df_non_empty['Cargo'].fillna(df_non_empty['quoted_words'])

<ipython-input-125-c97030beda81>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['Cargo'] = np.nan
<ipython-input-125-c97030beda81>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['Cargo']=df_non_empty['Cargo'].fillna(df_non_empty['combined_titular'])
<ipython-input-125-c97030beda81>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [126]:
df_empty_filtered['Cargo'] = np.nan
df_empty_filtered['Cargo'] = df_empty_filtered['Cargo'].fillna(df_empty_filtered['combined'])
df_empty_filtered['Cargo'] = df_empty_filtered['Cargo'].fillna(df_empty_filtered['processed_text'])

<ipython-input-126-4089d7dcdb8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['Cargo'] = np.nan
<ipython-input-126-4089d7dcdb8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['Cargo'] = df_empty_filtered['Cargo'].fillna(df_empty_filtered['combined'])
<ipython-input-126-4089d7dcdb8a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [127]:
df_empty_filtered

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Profesión,Genero,Reporta a,Secretaría,quoted_words,extracted_section,processed_text,contains_Titular,combined,Cargo
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,"[contadora pública, contadora]",F,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],de Directora de Tesorería y Contabilidad de la,Directora de Tesorería y Contabilidad,0,NaN,Directora de Tesorería y Contabilidad
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,"[ingeniero, ingeniero]",M,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA,[],de Director de Asuntos Satelitales de la,Director de Asuntos Satelitales,0,NaN,Director de Asuntos Satelitales
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,"[científico, agente, técnico]",M,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],,,0,NaN,
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,[licenciada],F,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN,[],de Directora de Carrera y Relaciones Laborale...,Directora de Carrera y Relaciones Laborales,0,NaN,Directora de Carrera y Relaciones Laborales
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,[licenciada],F,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,[],de Directora de Cooperación Internacional de ...,Directora de Cooperación Internacional,0,NaN,Directora de Cooperación Internacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,[doctora],F,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,[],de Directora de Contratación y Ejecución de O...,Directora de Contratación y Ejecución de Obras...,0,NaN,Directora de Contratación y Ejecución de Obras...
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,"[señor, informático]",M,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],,,0,NaN,
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,"[señora, analista, analista]",F,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL...",[],Nivel C – Grado 0 del Convenio Colectivo de T...,Nivel C – Grado 0 del Convenio Colectivo de Tr...,0,NaN,Nivel C – Grado 0 del Convenio Colectivo de Tr...
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,"[licenciada, técnica]",F,UNIDAD GABINETE DE ASESORES,UNIDAD GABINETE DE ASESORES,[],d

###Mapeando resultados

In [128]:
#Creamos llaves de identificación unicas a cada observación
df_filtrado_designaciones['llave_identificacion'] = df_filtrado_designaciones['DNI Extraido'] + ' ' + df_filtrado_designaciones['Parrafos_filtrados']
df_non_empty['llave_identificacion'] = df_non_empty['DNI Extraido'] + ' ' + df_non_empty['Parrafos_filtrados']
df_empty_filtered['llave_identificacion'] = df_empty_filtered['DNI Extraido'] + ' ' + df_empty_filtered['Parrafos_filtrados']

<ipython-input-128-ace31924c106>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty['llave_identificacion'] = df_non_empty['DNI Extraido'] + ' ' + df_non_empty['Parrafos_filtrados']
<ipython-input-128-ace31924c106>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty_filtered['llave_identificacion'] = df_empty_filtered['DNI Extraido'] + ' ' + df_empty_filtered['Parrafos_filtrados']


In [129]:
diccionario_not_empty = df_non_empty.set_index('llave_identificacion')['Cargo'].to_dict()
diccionario_empty = df_empty_filtered.set_index('llave_identificacion')['Cargo'].to_dict()

In [130]:
df_filtrado_designaciones['Cargo'] = df_filtrado_designaciones['llave_identificacion'].map(diccionario_not_empty)
df_filtrado_designaciones['Cargoleti'] = df_filtrado_designaciones['llave_identificacion'].map(diccionario_empty)

df_filtrado_designaciones['Cargo']=df_filtrado_designaciones['Cargo'].fillna(df_filtrado_designaciones['Cargoleti'])

df_filtrado_designaciones = df_filtrado_designaciones.drop(['Cargoleti'], axis=1)

In [131]:
df_filtrado_designaciones.to_excel('df_cargos_extraidos_0.xlsx', index=False)

In [132]:
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Planta Permanente,Venc_Desig_prorr,Grado y Nivel,Profesión,Genero,Reporta a,Secretaría,quoted_words,llave_identificacion,Cargo
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,False,2024-03-30,Nivel B - Grado 0,"[contadora pública, contadora]",F,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],10003393 ARTÍCULO 1º.- Dase por designada con ...,Directora de Tesorería y Contabilidad
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,False,2021-04-12,Nivel B - Grado 0,"[ingeniero, ingeniero]",M,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA,[],10068119 ARTÍCULO 1º.- Dase por designado con ...,Director de Asuntos Satelitales
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,True,NaT,Nivel A - Grado 8,"[científico, agente, técnico]",M,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,False,2021-10-16,Nivel B - Grado 0,[licenciada],F,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN,[],10125550 ARTÍCULO 1°.- Dase por designada con ...,Directora de Carrera y Relaciones Laborales
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,True,2021-02-09,Nivel B - Grado 11,[licenciada],F,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,[],10134579 ARTÍCULO 1°.- Desígnase transitoriame...,Directora de Cooperación Internacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,False,2023-12-04,Nivel B - Grado 0,[doctora],F,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,[],94126199 ARTÍCULO 2°.- Desígnase con carácter ...,Directora de Contratación y Ejecución de Obras...
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,False,2023-12-23,Nivel C - Grado 0,"[señor, informático]",M,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],94228607 ARTÍCULO 2º.- Desígnase con carácter ...,
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,False,2022-09-14,Nivel C - Grado 0,"[señora, analista, analista]",F,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL...",[],94743066 ARTÍCULO 2º.- Desígnase con carácter ...,Nivel C – Grado 0 del Convenio Colectivo de Tr...
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚB

In [133]:
df_filtrado_designaciones['last_character'] = df_filtrado_designaciones['Cargo'].apply(lambda x: x[-1] if pd.notnull(x) and len(x) > 0 else None)

In [134]:
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Venc_Desig_prorr,Grado y Nivel,Profesión,Genero,Reporta a,Secretaría,quoted_words,llave_identificacion,Cargo,last_character
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,2024-03-30,Nivel B - Grado 0,"[contadora pública, contadora]",F,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],10003393 ARTÍCULO 1º.- Dase por designada con ...,Directora de Tesorería y Contabilidad,d
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,2021-04-12,Nivel B - Grado 0,"[ingeniero, ingeniero]",M,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA,[],10068119 ARTÍCULO 1º.- Dase por designado con ...,Director de Asuntos Satelitales,s
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,NaT,Nivel A - Grado 8,"[científico, agente, técnico]",M,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",,None
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,2021-10-16,Nivel B - Grado 0,[licenciada],F,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN,[],10125550 ARTÍCULO 1°.- Dase por designada con ...,Directora de Carrera y Relaciones Laborales,s
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,2021-02-09,Nivel B - Grado 11,[licenciada],F,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,[],10134579 ARTÍCULO 1°.- Desígnase transitoriame...,Directora de Cooperación Internacional,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,2023-12-04,Nivel B - Grado 0,[doctora],F,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,[],94126199 ARTÍCULO 2°.- Desígnase con carácter ...,Directora de Contratación y Ejecución de Obras...,e
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,2023-12-23,Nivel C - Grado 0,"[señor, informático]",M,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],94228607 ARTÍCULO 2º.- Desígnase con carácter ...,,None
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,2022-09-14,Nivel C - Grado 0,"[señora, analista, analista]",F,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL...",[],94743066 ARTÍCULO 2º.- Desígnase con carácter ...,Nivel C – Grado 0 del Convenio Colectivo de Tr...,°
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,2021-11-06,Nivel 

In [135]:
df_filtrado_designaciones['non_letter'] = df_filtrado_designaciones['last_character'].apply(lambda x: 1 if x is not None and not x.isalpha() else 0)

In [136]:
df_filtrado_designaciones_1 = df_filtrado_designaciones[df_filtrado_designaciones['non_letter'] == 1]
df_filtrado_designaciones_1

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Grado y Nivel,Profesión,Genero,Reporta a,Secretaría,quoted_words,llave_identificacion,Cargo,last_character,non_letter
19,10690278,1,ARTÍCULO 1º.- Dase por designado con carácter ...,2019-12-19,2021-09-27,Designación,Renuncia,Decisión Administrativa 1871/2020,Resolución 3031/2021,MINISTERIO DE EDUCACIÓN,...,Nivel A - Grado 0,[arquitecto],M,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],10690278 ARTÍCULO 1º.- Dase por designado con ...,"Director General de Infraestructura,",",",1
81,12009735,0,ARTÍCULO 6°.- Desígnase en el cargo de Titular...,2020-04-06,2020-04-06,Designación,Designación,Resolución 66/2020,Resolución 66/2020,"MINISTERIO DE RELACIONES EXTERIORES, COMERCIO ...",...,None,[señor],M,UNIDAD DE EVALUACIÓN DE LAS POLÍTICAS PÚBLICAS...,UNIDAD DE EVALUACIÓN DE LAS POLÍTICAS PÚBLICAS...,[UNIDAD DE EVALUACIÓN DE LAS POLÍTICAS PÚBLICA...,12009735 ARTÍCULO 6°.- Desígnase en el cargo d...,Titular de la Unidad Ejecutora Especial Tempor...,"""",1
88,12114763,0,ARTÍCULO 1º.- Dáse por designado con carácter ...,2020-02-11,2020-02-11,Designación,Designación,Decisión Administrativa 734/2020,Decisión Administrativa 734/2020,"MINISTERIO DE RELACIONES EXTERIORES, COMERCIO ...",...,Nivel A - Grado 0,[arquitecto],M,SUBSECRETARÍA DE ASUNTOS NACIONALES,SUBSECRETARÍA DE ASUNTOS NACIONALES,[],12114763 ARTÍCULO 1º.- Dáse por designado con ...,"Director de Asuntos Nacionales,",",",1
124,12600015,0,ARTÍCULO 5º.- Desígnase en el cargo de Titular...,2022-03-14,2022-03-14,Designación,Designación,Resolución 122/2022,Resolución 122/2022,JEFATURA DE GABINETE DE MINISTROS,...,None,[licenciado],M,UNIDAD EJECUTORA ESPECIAL TEMPORARIA,UNIDAD EJECUTORA ESPECIAL TEMPORARIA,[RESILIENCIA ARGENTINA],12600015 ARTÍCULO 5º.- Desígnase en el cargo d...,Titular de la Unidad Ejecutora Especial Tempor...,"""",1
141,12713953,0,ARTÍCULO 2°.- Dáse por designado con carácter ...,2020-03-06,2020-03-06,Designación,Designación,Decisión Administrativa 712/2020,Decisión Administrativa 712/2020,MINISTERIO DE SEGURIDAD,...,Nivel A - Grado 0,[señor],M,SUBSECRETARÍA DE PROGRAMACIÓN FEDERAL Y ARTICU...,SECRETARÍA DE ARTICULACIÓN FEDERAL DE LA SEGUR...,[],12713953 ARTÍCULO 2°.- Dáse por designado con ...,Director de Relaciones Institucionales y Contr...,",",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,36648691,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2021-11-01,2021-11-01,Designación,Designación,Decisión Administrativa 17/2022,Decisión Administrativa 17/2022,MINISTERIO DE SEGURIDAD,...,Nivel D - Grado 0,[señor],M,SECRETARÍA DE SEGURIDAD Y POLITÍCA CRIMINAL,SECRETARÍA DE SEGURIDAD Y POLITÍCA CRIMINAL,[],36648691 ARTÍCULO 1º.- Dase por designado con ...,Nivel D - Grado 0 del Convenio Colectivo de Tr...,°,1
4130,38786482,0,ARTÍCULO 1°.- Desígnase con carácter transitor...,2022-03-16,2022-03-16,Designación,Designación,Decisión Administrativa 287/2022,Decisión Administrativa 287/2022,MINISTERIO DE DESARROLLO PRODUCTIVO,...,Nivel B - Grado 0,[señor],M,DIRECCIÓN NACIONAL DE REGLAMENTOS TÉCNICOS,SECRETARÍA DE COMERCIO INTERIOR,[],38786482 ARTÍCULO 1°.- Desígnase con carácter ...,Coordinador de Seguridad de los Bienes Comerci...,",",1
4170,42231463,0,ARTÍCULO 1º.- Desígnase con carácter transitor...,2023-03-06,2023-03-06,Designación,Designación,Decisión Administrativa 168/2023,Decisión Administrativa 168/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,Nivel E - Grado 0,[señor],M,DIRECCIÓN NACIONAL DEL REGISTRO NACIONAL DE RE...,SUBSECRETARÍA DE ASUNTOS REGISTRALES,[],42231463 ARTÍCULO 1º.- Desígnase con carácter ...,Nivel E – Grado 0 del Convenio Colectivo de Tr...,°,1
4198,7062054,0,ARTÍCULO 1°.- Dase por designado al señor Migu...,2020-03-11,2020-03-11,Designación,Designación,Decisión Administrativa 1015/2020,Decisión Administrativa 1015/2020,MINISTERIO DE EDU

In [137]:
def count_de_la(s):
    return s.count('de la') if pd.notnull(s) else 0
df_filtrado_designaciones['de_la_count'] = df_filtrado_designaciones['Cargo'].apply(count_de_la)

In [138]:
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Profesión,Genero,Reporta a,Secretaría,quoted_words,llave_identificacion,Cargo,last_character,non_letter,de_la_count
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,"[contadora pública, contadora]",F,DIRECCIÓN GENERAL DE ADMINISTRACIÓN,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],10003393 ARTÍCULO 1º.- Dase por designada con ...,Directora de Tesorería y Contabilidad,d,0,0
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,"[ingeniero, ingeniero]",M,"DIRECCIÓN NACIONAL DE PLANEAMIENTO, CONVERGENC...",SECRETARÍA DE INNOVACIÓN PÚBLICA,[],10068119 ARTÍCULO 1º.- Dase por designado con ...,Director de Asuntos Satelitales,s,0,0
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,"[científico, agente, técnico]",M,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",,None,0,0
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,[licenciada],F,DIRECCIÓN GENERAL DE RECURSOS HUMANOS,SECRETARÍA DE COORDINACIÓN,[],10125550 ARTÍCULO 1°.- Dase por designada con ...,Directora de Carrera y Relaciones Laborales,s,0,0
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,[licenciada],F,DIRECCIÓN NACIONAL DE COOPERACIÓN Y FINANCIAMI...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,[],10134579 ARTÍCULO 1°.- Desígnase transitoriame...,Directora de Cooperación Internacional,l,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,[doctora],F,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,[],94126199 ARTÍCULO 2°.- Desígnase con carácter ...,Directora de Contratación y Ejecución de Obras...,e,0,0
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,"[señor, informático]",M,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],94228607 ARTÍCULO 2º.- Desígnase con carácter ...,,None,0,0
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,"[señora, analista, analista]",F,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL...",[],94743066 ARTÍCULO 2º.- Desígnase con carácter ...,Nivel C – Grado 0 del Convenio Colectivo de Tr...,°,1,0
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,"[licenciada, técnica]",F,UNIDAD GABINETE DE ASESORES,UNIDAD GABINETE DE ASESORES,[],95806583 ARTÍCULO 2°.- Desígnase con carácter ...,Directora General de Coordinación Técnica e In...,l,0,0


In [139]:
df_filtrado_designaciones['first_word'] = df_filtrado_designaciones['Cargo'].apply(lambda x: x.split()[0] if pd.notnull(x) and len(x.split()) > 0 else None)

In [140]:
df_filtrado_designaciones['word_count'] = df_filtrado_designaciones['Cargo'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)

In [141]:
df_filtrado_designaciones.to_excel('df_cargos_extraidos_0.xlsx', index=False)

###Traemos excel con resoluciones de boletin oficial corregidas manualmente

In [158]:
data=pd.read_excel("df_cargos_extraidos_0 (2) (1).xlsx")
df_cargos_corregidos=pd.DataFrame(data)

In [166]:
df_cargos_corregidos

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Reporta a,Secretaría,quoted_words,llave_identificacion,Cargo,last_character,non_letter,de_la_count,first_word,word_count
0,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",Investigador con orientación en biología marin...,NaN,0,0,NaN,0
1,10201331,1,"ARTÍCULO 1°.- Desígnase, a partir del 10 de di...",2019-12-10,2021-09-21,Designación,Renuncia,Decreto 18/2019,Decreto 629/2021,"MINISTERIO DE CIENCIA, TECNOLOGÍA E INNOVACIÓN",...,NaN,"MINISTERIO DE CIENCIA, TECNOLOGÍA E INNOVACIÓN",[],"10201331 ARTÍCULO 1°.- Desígnase, a partir del...","Ministro de Ciencia, Tecnología e Innovación",NaN,0,0,NaN,0
2,10400346,1,ARTÍCULO 1°.- Desígnase en el cargo de Ministr...,2022-10-13,2023-12-10,Designación,Renuncia,Decreto 688/2022,Decreto 706/2023,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",...,NaN,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",[],10400346 ARTÍCULO 1°.- Desígnase en el cargo d...,"Ministra de Trabajo, Empleo y Seguridad Social",NaN,0,0,NaN,0
3,10520311,0,ARTÍCULO 7°.- Desígnase a Luis J. C. LEFEVRE (...,2021-04-27,2021-04-27,Designación,Designación,Resolución 207/2021,Resolución 207/2021,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",...,NaN,"MINISTERIO DE TRABAJO, EMPLEO Y SEGURIDAD SOCIAL",[],10520311 ARTÍCULO 7°.- Desígnase a Luis J. C. ...,Secretario Técnico,NaN,0,0,NaN,0
4,11176760,0,ARTÍCULO 1º.- Dase por asignada con carácter t...,2020-10-14,2020-10-14,Designación,Designación,Resolución 682/2021,Resolución 682/2021,JEFATURA DE GABINETE DE MINISTROS,...,DIRECCIÓN DE CONTRATACIONES Y SUMINISTROS,SECRETARÍA DE COORDINACIÓN ADMINISTRATIVA,[],11176760 ARTÍCULO 1º.- Dase por asignada con c...,Coordinadora de Bienes y Contrataciones,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243,36662501,0,ARTÍCULO 1°.- Desígnase a Ana Melisa FARFAN (D...,2022-08-04,2022-08-04,Designación,Designación,Resolución 493/2022,Resolución 493/2022,JEFATURA DE GABINETE DE MINISTROS,...,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS,[],36662501 ARTÍCULO 1°.- Desígnase a Ana Melisa ...,ASISTENTE DE SOPORTE ADMINISTRATIVO,e,0,1,ASISTENTE,47
4244,29972524,0,ARTÍCULO 1°.- Desígnase a Nicolás Alberto ÁLVA...,2022-11-02,2022-11-02,Designación,Designación,Resolución 810/2022,Resolución 810/2022,JEFATURA DE GABINETE DE MINISTROS,...,JEFATURA DE GABINETE DE MINISTROS,JEFATURA DE GABINETE DE MINISTROS,[],29972524 ARTÍCULO 1°.- Desígnase a Nicolás Alb...,TÉCNICO ESPECIALISTA EN COMINICACIÓN INSTITUCI...,e,0,1,TÉCNICO,48
4245,27823886,0,ARTÍCULO 1º.- Dase por designada con carácter ...,2023-05-01,2023-05-01,Designación,Designación,Decisión Administrativa 368/2023,Decisión Administrativa 368/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],27823886 ARTÍCULO 1º.- Dase por designada con ...,Coordinadora Nacional del PROGRAMA DE COORDINA...,O,0,0,Nivel,49
4246,22083698,0,"ARTÍCULO 2°.- Desígnase, a partir del día prim...",2022-03-22,2022-03-22,Designación,Designación,Resolución 716/2022,Resolución 716/2022,MINISTERIO DE SEGURIDAD,...,DIRECCIÓN DE RESPUESTA,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO DE LA J...,[],"22083698 ARTÍCULO 2°.- Desígnase, a partir del...",REFERENTE DE OPERACIONES,S,0,0,Nivel,51


In [159]:
df_cargos_corregidos["llave_identificacion"]=df_cargos_corregidos["llave_identificacion"].astype(str)
df_filtrado_designaciones['llave_identificacion']=df_filtrado_designaciones['llave_identificacion'].astype(str)

In [162]:
diccionario_cargos = df_cargos_corregidos.set_index('llave_identificacion')['Cargo'].to_dict()

In [167]:
df_filtrado_designaciones['Cargo_arreglado'] = df_filtrado_designaciones['llave_identificacion'].map(diccionario_cargos)

In [168]:
df_filtrado_designaciones

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,Secretaría,quoted_words,llave_identificacion,Cargo,last_character,non_letter,de_la_count,first_word,word_count,Cargo_arreglado
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,SUBSECRETARÍA DE GESTIÓN ADMINISTRATIVA,[],10003393 ARTÍCULO 1º.- Dase por designada con ...,Directora de Tesorería y Contabilidad,d,0,0,Directora,5,Directora de Tesorería y Contabilidad
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,SECRETARÍA DE INNOVACIÓN PÚBLICA,[],10068119 ARTÍCULO 1º.- Dase por designado con ...,Director de Asuntos Satelitales,s,0,0,Director,4,Director de Asuntos Satelitales
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,SECRETARÍA DE GESTIÓN Y EMPLEO PÚBLICO,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",,None,0,0,None,0,Investigador con orientación en biología marin...
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,SECRETARÍA DE COORDINACIÓN,[],10125550 ARTÍCULO 1°.- Dase por designada con ...,Directora de Carrera y Relaciones Laborales,s,0,0,Directora,6,Directora de Carrera y Relaciones Laborales
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,SECRETARÍA DE COORDINACIÓN INTERJURISDICCIONAL,[],10134579 ARTÍCULO 1°.- Desígnase transitoriame...,Directora de Cooperación Internacional,l,0,0,Directora,4,Directora de Cooperación Internacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,DIRECCIÓN NACIONAL DE DESARROLLO DE OBRAS DE T...,[],94126199 ARTÍCULO 2°.- Desígnase con carácter ...,Directora de Contratación y Ejecución de Obras...,e,0,0,Directora,9,Directora de Contratación y Ejecución de Obras...
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,[],94228607 ARTÍCULO 2º.- Desígnase con carácter ...,,None,0,0,None,0,responsable de Gestión de Proyectos de Desarro...
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,"SECRETARÍA DE COORDINACIÓN, BIENESTAR, CONTROL...",[],94743066 ARTÍCULO 2º.- Desígnase con carácter ...,Nivel C – Grado 0 del Convenio Colectivo de Tr...,°,1,0,Nivel,25,Analista Administrativa
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,UNIDAD GABINETE DE ASESORES,[],95806583 ARTÍCULO 2°.- Desígnase con carácter ...,Directora General de Coordinación Técnica e In...,l,0,0,Directora,7,Directora General de Coordinación Técnica e In...


In [169]:
#1 significa que cargo no fue arreglado, 0 que si
df_filtrado_designaciones['Cargo arreglado?'] = (df_filtrado_designaciones['Cargo'].fillna('NaN') == df_filtrado_designaciones['Cargo_arreglado'].fillna('NaN')).astype(int)

In [171]:
df_filtrado_designaciones["Cargo arreglado?"].value_counts()

,count
Cargo arreglado?,
1,3606
0,633


In [179]:
df_filtrado_designaciones_cargo_arreglado=df_filtrado_designaciones[df_filtrado_designaciones["Cargo arreglado?"]==0]

In [180]:
df_filtrado_designaciones_cargo_arreglado.columns

Index(['DNI Extraido', 'Tiene_mas_de_un_evento', 'Parrafos_filtrados',
       'first_date', 'ult_date', 'first_class', 'last_class', 'first_res',
       'last_res', 'first_min', 'last_min', 'Nombre Completo', 'CUIT', 'Edad',
       'Provincia', 'Ciudad', 'Via Excepción', 'Designación Transitoria',
       'Planta Permanente', 'Venc_Desig_prorr', 'Grado y Nivel', 'Profesión',
       'Genero', 'Reporta a', 'Secretaría', 'quoted_words',
       'llave_identificacion', 'Cargo', 'last_character', 'non_letter',
       'de_la_count', 'first_word', 'word_count', 'Cargo_arreglado',
       'Cargo arreglado?'],
      dtype='object')

In [181]:
df_filtrado_designaciones_cargo_arreglado=df_filtrado_designaciones_cargo_arreglado[['DNI Extraido', 'Parrafos_filtrados', 'Cargo', 'Cargo_arreglado']]

In [186]:
palabras_eliminar = ['ad honórem', 'ad-honórem', 'ad- honorem']
df_filtrado_designaciones_cargo_arreglado = df_filtrado_designaciones_cargo_arreglado[~df_filtrado_designaciones_cargo_arreglado['Parrafos_filtrados'].str.contains('|'.join(palabras_eliminar), case=False, na=False)]

In [189]:
df_filtrado_designaciones_cargo_arreglado.to_excel("Cargos_arreglados_manualmente.xlsx", index=False)

###Arreglamos cargos y df

In [191]:
df_filtrado_designaciones['Cargo']=df_filtrado_designaciones['Cargo_arreglado']

In [194]:
df_filtrado_designaciones.columns

Index(['DNI Extraido', 'Tiene_mas_de_un_evento', 'Parrafos_filtrados',
       'first_date', 'ult_date', 'first_class', 'last_class', 'first_res',
       'last_res', 'first_min', 'last_min', 'Nombre Completo', 'CUIT', 'Edad',
       'Provincia', 'Ciudad', 'Via Excepción', 'Designación Transitoria',
       'Planta Permanente', 'Venc_Desig_prorr', 'Grado y Nivel', 'Profesión',
       'Genero', 'Reporta a', 'Secretaría', 'quoted_words',
       'llave_identificacion', 'Cargo', 'last_character', 'non_letter',
       'de_la_count', 'first_word', 'word_count', 'Cargo_arreglado',
       'Cargo arreglado?'],
      dtype='object')

In [193]:
df_filtrado_designaciones=df_filtrado_designaciones.drop(['last_character', 'non_letter', 'de_la_count', 'first_word', 'word_count'], axis=1)

,DNI Extraido,Tiene_mas_de_un_evento,Parrafos_filtrados,first_date,ult_date,first_class,last_class,first_res,last_res,first_min,...,quoted_words,llave_identificacion,Cargo,last_character,non_letter,de_la_count,first_word,word_count,Cargo_arreglado,Cargo arreglado?
0,10003393,1,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,2023-10-02,Designación,Prorroga,Decisión Administrativa 828/2020,Resolución 2120/2023,MINISTERIO DE SALUD,...,[],10003393 ARTÍCULO 1º.- Dase por designada con ...,Directora de Tesorería y Contabilidad,d,0,0,Directora,5,Directora de Tesorería y Contabilidad,1
1,10068119,0,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,2020-10-14,Designación,Designación,Decisión Administrativa 658/2021,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,...,[],10068119 ARTÍCULO 1º.- Dase por designado con ...,Director de Asuntos Satelitales,s,0,0,Director,4,Director de Asuntos Satelitales,1
2,10077705,0,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,2022-03-22,Designación,Designación,Resolución 1613/2023,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,...,[],"10077705 ARTÍCULO 1°.- Desígnase, a partir del...",Investigador con orientación en biología marin...,None,0,0,None,0,Investigador con orientación en biología marin...,0
3,10125550,1,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,2021-04-19,Designación,Prorroga,Decisión Administrativa 1443/2020,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,...,[],10125550 ARTÍCULO 1°.- Dase por designada con ...,Directora de Carrera y Relaciones Laborales,s,0,0,Directora,6,Directora de Carrera y Relaciones Laborales,1
4,10134579,0,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,2020-08-13,Designación,Designación,Decisión Administrativa 1451/2020,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,...,[],10134579 ARTÍCULO 1°.- Desígnase transitoriame...,Directora de Cooperación Internacional,l,0,0,Directora,4,Directora de Cooperación Internacional,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,94126199,1,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,2023-06-07,Designación,Prorroga,Decisión Administrativa 755/2022,Resolución 335/2023,MINISTERIO DE TRANSPORTE,...,[],94126199 ARTÍCULO 2°.- Desígnase con carácter ...,Directora de Contratación y Ejecución de Obras...,e,0,0,Directora,9,Directora de Contratación y Ejecución de Obras...,1
4235,94228607,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,2023-06-26,Designación,Designación,Decisión Administrativa 510/2023,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,...,[],94228607 ARTÍCULO 2º.- Desígnase con carácter ...,responsable de Gestión de Proyectos de Desarro...,None,0,0,None,0,responsable de Gestión de Proyectos de Desarro...,0
4236,94743066,0,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,2022-03-18,Designación,Designación,Decisión Administrativa 296/2022,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,...,[],94743066 ARTÍCULO 2º.- Desígnase con carácter ...,Analista Administrativa,°,1,0,Nivel,25,Analista Administrativa,0
4237,95806583,0,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,2021-05-10,Designación,Designación,Decisión Administrativa 453/2021,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,...,[],95806583 ARTÍCULO 2°.- Desígnase con carácter ...,Directora General de Coordinación Técnica e In...,l,0,0,Directora,7,Directora General de Coordinación Técnica e In...,1


#Mapeando Datos de Movimientos

In [ ]:
data=pd.read_excel("dataset_bureaucrats_V6_SV.xlsx")
df_movimientos=pd.DataFrame(data)
df_movimientos

In [ ]:
df_movimientos['national_id'] = df_movimientos['national_id'].astype(str)
df_filtrado_designaciones['DNI Extraido'] = df_filtrado_designaciones['DNI Extraido'].astype(str)

In [ ]:
diccionario_movimiento = df_movimientos.set_index('national_id')['affiliation'].to_dict()
diccionario_movimiento_SV_AJ=df_movimientos.set_index('parrafos_filtrados')['ministry_r'].to_dict()
diccionario_movimiento_SV_AJ_2=df_movimientos.set_index('national_id')['comments'].to_dict()

In [ ]:
df_filtrado_designaciones['affiliation'] = df_filtrado_designaciones['DNI Extraido'].map(diccionario_movimiento)
df_filtrado_designaciones['ministry_r'] = df_filtrado_designaciones['Parrafos_filtrados'].map(diccionario_movimiento_SV_AJ)
df_filtrado_designaciones['Comentarios'] = df_filtrado_designaciones['DNI Extraido'].map(diccionario_movimiento_SV_AJ_2)

In [ ]:
df_filtrado_designaciones.columns

In [ ]:
df_filtrado_designaciones

#Agregando primer palabra cargo
Función para extraer la primer palabra de la variable "cargo"

In [ ]:
def extraer_primera_palabra(texto):
    if pd.isna(texto) or texto.strip() == "":
        return np.nan
    return texto.split()[0]

df_filtrado_designaciones['Primera Palabra Cargo'] = df_filtrado_designaciones['Cargo Extraido'].apply(extraer_primera_palabra)

In [ ]:
df_filtrado_designaciones

#Reordenando el DF

In [ ]:
nuevo_orden = [
    'DNI Extraido', 'CUIT', 'Nombre Completo', 'Edad', 'Genero', 'Profesión', 'Provincia', 'Ciudad',
    'Parrafos_filtrados', 'first_date', 'first_class', 'Cargo Extraido', 'Via Excepción',
    'Designación Transitoria', 'Planta Permanente', 'Grado y Nivel', 'first_res', 'Reporta a',
    'Secretaría', 'first_min', 'last_class', 'last_res', 'last_min', 'ult_date', 'Venc_Desig_prorr',
    'affiliation', 'ministry_r', 'Comentarios', 'Primera Palabra Cargo'
]

# Reordenar las columnas en el DataFrame
df_filtrado_designaciones = df_filtrado_designaciones[nuevo_orden]

In [ ]:
df_filtrado_designaciones

In [ ]:
df_filtrado_designaciones = df_filtrado_designaciones.rename(columns={'Parrafos_filtrados': 'first_art'})

In [ ]:
df_filtrado_designaciones

,DNI Extraido,CUIT,Nombre Completo,Edad,Genero,Profesión,Provincia,Ciudad,first_art,first_date,...,first_min,last_class,last_res,last_min,ult_date,Venc_Desig_prorr,affiliation,ministry_r,Comentarios,Primera Palabra Cargo
0,10003393,27-10003393-9,Josefina Magdalena Meza,72.0,F,"[contadora pública, contadora]",Formosa,Formosa,ARTÍCULO 1º.- Dase por designada con carácter ...,2020-02-01,...,MINISTERIO DE SALUD,Prorroga,Resolución 2120/2023,MINISTERIO DE SALUD,2023-10-02,2024-03-30,Partido Peronista,Health,['nan'],Directora
1,10068119,20-10068119-7,Ricardo Luis Teran,71.0,M,"[ingeniero, ingeniero]",Santa Fe,No Encontrado,ARTÍCULO 1º.- Dase por designado con carácter ...,2020-10-14,...,JEFATURA DE GABINETE DE MINISTROS,Designación,Decisión Administrativa 658/2021,JEFATURA DE GABINETE DE MINISTROS,2020-10-14,2021-04-12,Partido Peronista,Chief of Staff,['nan'],Director
2,10077705,20-10077705-4,Oscar Horacio Padin,72.0,M,"[científico, agente, técnico]",Ciudad De Buenos Aires,Palermo,"ARTÍCULO 1°.- Desígnase, a partir del dictado ...",2022-03-22,...,MINISTERIO DE ECONOMÍA,Designación,Resolución 1613/2023,MINISTERIO DE ECONOMÍA,2022-03-22,NaT,De carrera,Economy,NaN,Investigador
3,10125550,27-10125550-1,Elsa Liliana Garcia,72.0,F,[licenciada],Ciudad De Buenos Aires,Caballito,ARTÍCULO 1°.- Dase por designada con carácter ...,2020-07-01,...,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,Prorroga,Resolución 109/2021,MINISTERIO DE DESARROLLO TERRITORIAL Y HÁBITAT,2021-04-19,2021-10-16,De carrera,Territorial Development and Housing,['nan'],Directora
4,10134579,27-10134579-9,Victoria Lichtschein,72.0,F,[licenciada],Ciudad De Buenos Aires,Palermo,"ARTÍCULO 1°.- Desígnase transitoriamente, a pa...",2020-08-13,...,JEFATURA DE GABINETE DE MINISTROS,Designación,Decisión Administrativa 1451/2020,JEFATURA DE GABINETE DE MINISTROS,2020-08-13,2021-02-09,De carrera,Chief of Staff,Es lanta permanente,Directora
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243,94126199,27-94126199-5,Maria Elisa Osano Cerdeña,36.0,F,[doctora],Ciudad De Buenos Aires,Villa Crespo,ARTÍCULO 2°.- Desígnase con carácter transitor...,2022-08-01,...,MINISTERIO DE TRANSPORTE,Prorroga,Resolución 335/2023,MINISTERIO DE TRANSPORTE,2023-06-07,2023-12-04,Partido Peronista,Transportation,NaN,Directora
4244,94228607,20-94228607-5,Sergio Matias Yahari Vazquez,33.0,M,"[señor, informático]",Buenos Aires,Paso Del Rey,ARTÍCULO 2º.- Desígnase con carácter transitor...,2023-06-26,...,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,Designación,Decisión Administrativa 510/2023,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,2023-06-26,2023-12-23,NaN,Justice,NaN,responsable
4245,94743066,27-94743066-7,Teresa De Jesus Colman Zarate,29.0,F,"[señora, analista, analista]",Ciudad De Buenos Aires,No Encontrado,ARTÍCULO 2º.- Desígnase con carácter transitor...,2022-03-18,...,MINISTERIO DE SEGURIDAD,Designación,Decisión Administrativa 296/2022,MINISTERIO DE SEGURIDAD,2022-03-18,2022-09-14,NaN,Security,NaN,Analista
4246,95806583,27-95806583-9,Renee Carolina Cordova Malaver,44.0,F,"[licenciada, técnica]",No Encontrado,No Encontrado,ARTÍCULO 2°.- Desígnase con carácter transitor...,2021-05-10,...,MINISTERIO DE OBRAS PÚBLICAS,Designación,Decisión Administrativa 453/2021,MINISTERIO DE OBRAS PÚBLICAS,2021-05-10,2021-11-06,Partido Peronista,Infrastructure,['nan'],Directora


In [ ]:
df_filtrado_designaciones.to_excel('Borrador_burocrats_colapsado.xlsx', index=False)